In [31]:
# BALTIMORE 288500
debug = 1
city = "Pittsburgh" # Possible values: Baltimore, Washington DC, Detroit, Newark, St. Louis, Richmond, San Francisco, Cleveland, Philadelphia, Pittsburgh
g = 0

latest_year = 2022
earliest_year = 2012

required_overlap = 0.5

check_tract_and_city_boundaries = False
check_tract_consistency = False

cross_check_acs5_year_columns = False

show_water = False
show_roads = False

print('Importing packages...')
import os
import pandas
import numpy as np
import time
import geopandas
from pprint import pprint
from matplotlib import pyplot as plt
from shapely.ops import unary_union
from shapely.geometry import Point, box
import matplotlib.patches as mpatches
import requests
import json
from pprint import pprint
import re
import io

pandas.set_option('display.max_colwidth', None)


# Directories to look in
thisdir = os.getcwd() + '/' #'/home/idies/workspace/Storage/raddick/Baltimore/community_reinvestment_act/'
data_dir = '/home/idies/workspace/21cc/Data/'
loans_dir = data_dir + 'Community Reinvestment Act/'
jobs_dir = data_dir + 'LODES8/'

census_dir = data_dir + 'Census/ACS5/'

output_data_dir = thisdir + 'city_data/'
if (not(os.path.exists(output_data_dir))):
    os.makedirs(output_data_dir)

#baltimore_dir = thisdir + 'baltimore/'

census_shapefile_tiger_basedir = data_dir + 'Census/Shapefiles/TIGER/'

#shapefile_dir = census_shapefile_tiger_basedir + '{0:.0f}/TRACT/'.format(thisyear)

#acs5_basedir = '/home/idies/workspace/Temporary/raddick/census_scratch/acs5/'

code_lookup_dir = thisdir + 'code_guide_lookups/'
#inflation_dir = '/home/idies/workspace/Storage/raddick/Baltimore/community_reinvestment_act/datasets/inflation/'
#extrasdir = '/home/idies/workspace/Storage/raddick/census/extras/'


scale = 1

equal_area_epsg = 5070

os.chdir(thisdir)
g = 0 # global time

print('Now in directory: {0:}'.format(os.getcwd()))
print('ok')

Importing packages...
Now in directory: /home/idies/workspace/21cc/raddick/community_reinvestment_act
ok


In [2]:
s = time.time()
years = np.arange(latest_year, earliest_year-1, -1)

if (city == 'Baltimore'):
    thestate = 24
    state_abbrev = 'md'
    citycode = 4000
    baltimore_shapefile_dir = '/home/idies/workspace/Storage/raddick/Baltimore/shapefiles/'
    plotlimits = {'N': 39.38, 'S': 39.195, 'W': -76.745, 'E': -76.505}

elif (city == 'Washington DC'):
    thestate = 11  
    state_abbrev = 'dc'
    citycode = 50000
    plotlimits = {'N': 39.01, 'S': 38.79, 'E': -76.9, 'W': -77.13}

elif (city == 'Detroit'):
    thestate = 26   # state_codes_df[state_codes_df['STATE_NAME'] == 'Missouri']
    state_abbrev = 'mi'
    citycode = 22000
    plotlimits = {'N': 42.45, 'S': 42.25, 'E': -82.9, 'W': -83.3}
    
elif (city == 'Newark'):
    thestate = 34
    state_abbrev = 'nj'
    citycode = 51000
    plotlimits = {'N': 40.8, 'S': 40.6, 'E': -74.1, 'W': -74.3}

elif (city == 'St. Louis'):
    thestate = 29   # state_codes_df[state_codes_df['STATE_NAME'] == 'Missouri']
    state_abbrev = 'mo'
    citycode = 65000
    plotlimits = {'N': 38.8, 'S': 38.5, 'E': -90.1, 'W': -90.4}
    
elif (city == 'Richmond'):
    thestate = 51   # state_codes_df[state_codes_df['STATE_NAME'] == 'Missouri']
    state_abbrev = 'va'
    citycode = 67000
    plotlimits = {'N': 37.61, 'S': 37.43, 'E': -77.37, 'W': -77.61}
elif (city == 'San Francisco'):
    thestate = 6   # state_codes_df[state_codes_df['STATE_NAME'] == 'California']
    state_abbrev = 'ca'
    plotlimits = {'N': 37.9, 'S': 37.65, 'E': -122.2, 'W': -123.2}  # with Farallon Islands
    #plotlimits = {'N': 37.82, 'S': 37.7, 'E': -122.35, 'W': -122.53}  # without Farallon Islands
    citycode = 67000    
elif (city == 'Pittsburgh'):
    thestate = 42
    state_abbrev = 'pa'
    citycode = 61000
#    plotlimits = {'N': 40.55, 'S': 40.3, 'E': -79.85, 'W': -80.15}
    plotlimits = {'N': 40.5, 'S': 40.35, 'E': -79.85, 'W': -80.1}

elif (city == 'Cleveland'):
    thestate = 39
    state_abbrev = 'oh'
    citycode = 16000
    plotlimits = {'N': 41.65, 'S': 41.35, 'E': -81.5, 'W': -81.9}
    
elif (city == 'Philadelphia'):
    thestate = 42
    state_abbrev = 'pa'
    #thecounty = 101
    citycode = 60000
    plotlimits = {'N': 40.15, 'S': 39.85, 'E': -74.93, 'W': -75.3}
else:
    print('ERROR: Select city from list!')

# Initiailze water and roads whether we need them or not
water_gdf = None
roads_gdf = None
    
cityname_file = city.lower().replace(' ','_')
e = time.time()
g = g + (e-s)
print('Selected {0:}!'.format(city))

Selected Pittsburgh!


# Get water and roads if needed

In [3]:
# show_year = 2022
# water_gdf = None
# roads_gdf = None
# show_water = True
# show_roads = True

# if ((show_water) & (water_gdf is not None)):
#     s = time.time()
#     print('reading water files...')
#     water_dir = census_shapefile_tiger_basedir+"{0:.0f}/AREAWATER/{1:}/".format(show_year, state_abbrev)
#     #print(water_dir)
#     water_files = [water_dir+x for x in os.listdir(water_dir) if ('tl_{0:.0f}_{1:02d}'.format(show_year,thestate) in x) and (x[-3:] == 'shp')]
#     water_gdf = geopandas.GeoDataFrame()
#     for i in range(0, len(water_files)):
#         if ((np.mod(i,10) == 0) | (i == len(water_files)-1)):
#             print('\treading water file {0:,.0f} of {1:,.0f}...'.format(i+1, len(water_files)))
#         water_gdf = pandas.concat((water_gdf, geopandas.read_file(water_files[i])))
#     e = time.time()
#     g += e-s
#     print('Got {0:,.0f} bodies of water in {1:,.1f} seconds!'.format(len(water_gdf), e-s))
    

# if ((show_roads) & (roads_gdf is not None)):
#     s = time.time()
#     print('\n')
#     print('reading road files...')
#     road_dir = census_shapefile_tiger_basedir+"{0:.0f}/ROADS/{1:}/".format(show_year, state_abbrev)
#     road_files = [road_dir+x for x in os.listdir(road_dir) if ('tl_{0:.0f}_{1:02d}'.format(show_year,thestate) in x) and (x[-3:] == 'shp')]
#     roads_gdf = geopandas.GeoDataFrame()
#     for i in range(0, len(road_files)):
#         if ((np.mod(i,10) == 0) | (i == len(road_files)-1)):
#             print('\treading road file {0:,.0f} of {1:,.0f}...'.format(i+1, len(road_files)))
#         roads_gdf = pandas.concat((roads_gdf, geopandas.read_file(road_files[i])))
#     e = time.time()
#     g += e-s
#     print('Got {0:,.0f} roads in {1:,.1f} seconds!'.format(len(roads_gdf), e-s))
# print('Done!')

## Do city boundaries line up with census tract boundaries?

Baltimore: Yes

Washington, DC: Yes

Detroit: No

Cleveland: Almost

Newark: Yes

St. Louis: Yes

Richmond: No (the place shape changed slightly 2016 -> 2017, but tracts unchanged)

In [4]:
s = time.time()
#required_overlap = 0.5
#check_tract_and_city_boundaries = True

show_year = 2022

show_water = False
show_roads = False

xlim = [plotlimits['W'], plotlimits['E']]
ylim = [plotlimits['S'], plotlimits['N']]

# xlim = [plotlimits['W'], -77.47]
# ylim = [37.51, plotlimits['N']]

city_tracts_years_gdf = geopandas.GeoDataFrame()
partial_overlaps = {}

plot_geo = box(xlim[0], ylim[0], xlim[1], ylim[1])
if (show_water):
    plot_gdf = geopandas.GeoDataFrame(data=[[plot_geo]], columns=['geometry'], crs=water_gdf.crs)
else:
    plot_gdf = geopandas.GeoDataFrame(data=[[plot_geo]], columns=['geometry'], crs=city_tracts_years_gdf.crs)
    
if (show_water):
    water_here_gdf = geopandas.overlay(water_gdf, plot_gdf, how='intersection')
if (show_roads):
    roads_here_gdf = geopandas.overlay(roads_gdf, plot_gdf, how='intersection')


for thisyear in years:
    # Get tracts this year
    if (thisyear == 2010):
        state_tracts_this_year_gdf = geopandas.read_file(census_shapefile_tiger_basedir +'{0:.0f}/TRACT/tl_{0:.0f}_{1:02d}_tract10.shp'.format(thisyear, thestate))#.set_index('GEOID')
        for x in state_tracts_this_year_gdf.columns[:-1]:
            state_tracts_this_year_gdf = state_tracts_this_year_gdf.rename(columns = {x: x[:-2]})
        state_tracts_this_year_gdf = state_tracts_this_year_gdf.set_index('GEOID')
    else:
        state_tracts_this_year_gdf = geopandas.read_file(census_shapefile_tiger_basedir +'{0:.0f}/TRACT/tl_{0:.0f}_{1:02d}_tract.shp'.format(thisyear, thestate)).set_index('GEOID')
    state_tracts_this_year_gdf = state_tracts_this_year_gdf.sort_index()
    
    # Get places this year
    if (thisyear == 2010):
        state_places_this_year_gdf = geopandas.read_file(census_shapefile_tiger_basedir + '{0:.0f}/PLACE/tl_{0:.0f}_{1:02d}_place10.shp'.format(thisyear, thestate))#.set_index('GEOID')
        for x in state_places_this_year_gdf.columns[:-1]:
            state_places_this_year_gdf = state_places_this_year_gdf.rename(columns = {x: x[:-2]})
        state_places_this_year_gdf = state_places_this_year_gdf.set_index('GEOID')
    else:
        state_places_this_year_gdf = geopandas.read_file(census_shapefile_tiger_basedir + '{0:.0f}/PLACE/tl_{0:.0f}_{1:02d}_place.shp'.format(thisyear, thestate)).set_index('GEOID')
    state_places_this_year_gdf = state_places_this_year_gdf.sort_index()
    
    # Prepare to make plot later
    if (check_tract_and_city_boundaries):
        if (thisyear == show_year):
            state_tracts_show_year_gdf = state_tracts_this_year_gdf
            state_places_show_year_gdf = state_places_this_year_gdf
        
        
    print('{0:.0f}: found {1:,.0f} total census tracts and {2:,.0f} total places in {3:}...'.format(thisyear, len(state_tracts_this_year_gdf), len(state_places_this_year_gdf), state_abbrev))

    # Get this city only
    this_city_this_year_gdf = state_places_this_year_gdf[state_places_this_year_gdf['PLACEFP'] == '{0:05d}'.format(citycode)]
    
    if (len(this_city_this_year_gdf.geometry.values) == 1):
        city_year_geo = this_city_this_year_gdf.geometry.values[0]
        city_year_area_sq_m = this_city_this_year_gdf.to_crs(epsg=equal_area_epsg).geometry.area.values[0]
        print('\tMatching {0:} {1:.0f} data (city area = {2:,.1f} square km)...'.format(city, thisyear, city_year_area_sq_m/(1000**2)))
    else:
        print('wtf, this city has more than one geometry included?')
        break
    
    cnt = 0
    this_year_partial_overlap_list = []
    city_tracts_this_year_gdf = geopandas.GeoDataFrame(data=None, columns=state_tracts_this_year_gdf.columns, crs=state_tracts_this_year_gdf.crs, geometry='geometry')
    
    for ix, thisrow in state_tracts_this_year_gdf.iterrows():
# #         if (np.mod(cnt,100) == 0):
# #             print('\t\tMatching tract {0:,.0f} of {1:,.0f}...'.format(cnt, ntracts_this_state_year))
        if (thisrow.geometry.intersects(city_year_geo)):        
            intersection_geo = state_tracts_this_year_gdf.loc[ix].geometry.intersection(city_year_geo)
            pct_overlap = intersection_geo.area / thisrow.geometry.area
            if (pct_overlap >= 0.995):
                city_tracts_this_year_gdf = pandas.concat((city_tracts_this_year_gdf, state_tracts_this_year_gdf[state_tracts_this_year_gdf.index == ix]), axis=0)
            elif (pct_overlap >= 0.01):
                intersection_gdf = geopandas.GeoDataFrame(data=[[intersection_geo]], columns=['geometry'], crs=state_tracts_this_year_gdf.crs, geometry='geometry')
                tract_total_area_sq_m = state_tracts_this_year_gdf.to_crs(epsg=equal_area_epsg).loc[ix].geometry.area
                intersection_area_sq_m = intersection_gdf.to_crs(epsg=equal_area_epsg).geometry.apply(lambda x: x.area).values[0]
                pct_overlap_sq_m = intersection_area_sq_m / tract_total_area_sq_m        
                print('\n\t\tPARTIAL overlap found: County {0:} Tract {1:}\n\t\t\tTotal area {2:,.2f} sq km, overlap area {3:,.2f} sq km ({4:.1%})'.format(thisrow['COUNTYFP'], thisrow['TRACTCE'], tract_total_area_sq_m/1000000, intersection_area_sq_m/1000000, pct_overlap_sq_m))
                this_year_partial_overlap_list.append(ix)
                if (pct_overlap >= required_overlap):
                    city_tracts_this_year_gdf = pandas.concat((city_tracts_this_year_gdf, state_tracts_this_year_gdf[state_tracts_this_year_gdf.index == ix]), axis=0)
        cnt += 1
        
    city_tracts_this_year_gdf = city_tracts_this_year_gdf.assign(year = thisyear)
    city_tracts_years_gdf = pandas.concat((city_tracts_years_gdf, city_tracts_this_year_gdf), axis=0)
    
    #if (len(this_year_partial_overlap_list) > 0):
    partial_overlaps[thisyear] = this_year_partial_overlap_list
    
    print('\n')
    
city_tracts_years_gdf.loc[:, 'STATEFP'] = pandas.to_numeric(city_tracts_years_gdf['STATEFP'], errors='coerce')
city_tracts_years_gdf.loc[:, 'COUNTYFP'] = pandas.to_numeric(city_tracts_years_gdf['COUNTYFP'], errors='coerce')
city_tracts_years_gdf.loc[:, 'TRACTCE'] = pandas.to_numeric(city_tracts_years_gdf['TRACTCE'].apply(lambda x: '{0:}.{1:}'.format(x[0:-2],x[-2:])), errors='coerce')
for x in ['ALAND', 'AWATER', 'INTPTLAT','INTPTLON']:
    city_tracts_years_gdf.loc[:, x] = pandas.to_numeric(city_tracts_years_gdf[x], errors='coerce')

city_tracts_years_gdf = city_tracts_years_gdf.rename(columns={'COUNTYFP': 'county', 'TRACTCE': 'census_tract'})
city_tracts_years_gdf.index.name = 'GEOID'
city_tracts_years_gdf = city_tracts_years_gdf.reset_index()
city_tracts_years_gdf.loc[:, 'GEOID'] = city_tracts_years_gdf['GEOID'].apply(lambda x: '14000US'+x)
city_tracts_years_gdf = city_tracts_years_gdf.set_index(['GEOID','year'])

e = time.time()
g = g + (e-s)
print('{0:}: Found {1:,.0f} tract-years in {2:,.0f} seconds!'.format(city, len(city_tracts_years_gdf), e-s))

# PLOT!
if (check_tract_and_city_boundaries):
    s = time.time()
    print('\n')
    print('plotting tract vs city for {0:.0f}...'.format(show_year))
    
    partial_overlap_tracts_show_year_gdf = state_tracts_show_year_gdf.loc[partial_overlaps[show_year]]
    
    fig, ax = plt.subplots(1,1, figsize=(12,12))
    state_tracts_show_year_gdf.plot(color='none', edgecolor='black', ax=ax, linewidth=0.25)    
    state_places_show_year_gdf[state_places_show_year_gdf['PLACEFP'] == '{0:05d}'.format(citycode)].plot(ax=ax, color='none', edgecolor='yellow', lw=5)

    city_tracts_years_gdf.xs(show_year, level='year').plot(ax=ax, color='red', alpha=0.5, edgecolor='black', linewidth=0.5)
    #city_tracts_years_gdf.xs(2016, level='year').plot(ax=ax, color='green', alpha=0.5, edgecolor='black', linewidth=0.5)
        
    if (show_water):
        print('\tplotting water...')
        water_here_gdf.plot(ax=ax, color='blue')

    if (show_roads):
        print('\tplotting major roads...')
        roads_here_gdf[roads_here_gdf['RTTYP'] == 'I'].plot(ax=ax, color='purple', linewidth=5)
        roads_here_gdf[roads_here_gdf['RTTYP'] == 'U'].plot(ax=ax, color='purple', linewidth=3)
        roads_here_gdf[roads_here_gdf['RTTYP'] == 'S'].plot(ax=ax, color='purple', linewidth=2)
        print('\tplotting minor roads...')
        roads_here_gdf[roads_here_gdf['RTTYP'] == 'M'].plot(ax=ax, color='purple', linewidth=1)
        roads_here_gdf[roads_here_gdf['RTTYP'].isnull()].plot(ax=ax, color='purple', linewidth=0.5)
        
    if (len(partial_overlap_tracts_show_year_gdf) > 0):
        partial_overlap_tracts_show_year_gdf.plot(ax=ax, color='none', edgecolor='black', linewidth=3, alpha=0.5)
        for ix, thisrow in partial_overlap_tracts_show_year_gdf.iterrows():
            annotator = '{0:}'.format(np.float(thisrow['NAME']))
            ax.annotate(annotator, (thisrow.geometry.centroid.x, thisrow.geometry.centroid.y), (thisrow.geometry.centroid.x, thisrow.geometry.centroid.y), 
                        color='black', bbox=dict(fc='white', ec='black'))

    ax.set_xlim(xlim)
    ax.set_ylim(ylim)
        
    plt.title('{0:.0f}'.format(show_year), size=24)
    
    plt.show()
    e = time.time()
    g += e-s 

print('Done!')

2022: found 3,446 total census tracts and 1,888 total places in pa...
	Matching Pittsburgh 2022 data (city area = 151.1 square km)...


2021: found 3,446 total census tracts and 1,888 total places in pa...
	Matching Pittsburgh 2021 data (city area = 151.1 square km)...


2020: found 3,446 total census tracts and 1,888 total places in pa...
	Matching Pittsburgh 2020 data (city area = 151.1 square km)...


2019: found 3,218 total census tracts and 1,761 total places in pa...
	Matching Pittsburgh 2019 data (city area = 151.1 square km)...


2018: found 3,218 total census tracts and 1,761 total places in pa...
	Matching Pittsburgh 2018 data (city area = 151.1 square km)...


2017: found 3,218 total census tracts and 1,761 total places in pa...
	Matching Pittsburgh 2017 data (city area = 151.1 square km)...


2016: found 3,218 total census tracts and 1,761 total places in pa...
	Matching Pittsburgh 2016 data (city area = 151.1 square km)...


2015: found 3,218 total census tracts and 1,762 

# Which census tracts changed, when?

In [5]:
show_water = False
show_roads = False

xlim = [plotlimits['W'], plotlimits['E']]
ylim = [plotlimits['S'], plotlimits['N']]

xlim = [-79.97, -79.9]
ylim = [40.37, 40.43]

print('How many tracts per year?')
theyears = city_tracts_years_gdf.index.get_level_values(level='year').drop_duplicates().sort_values().tolist()
for thisyear in theyears:
    print('\t{0:}: found {1:,.0f} tracts!'.format(thisyear, len(city_tracts_years_gdf.xs(thisyear, level='year'))))

    
if (check_tract_consistency):
    plot_geo = box(xlim[0], ylim[0], xlim[1], ylim[1])
    plot_gdf = geopandas.GeoDataFrame(data=[[plot_geo]], columns=['geometry'], crs=city_tracts_years_gdf.crs)
    if (show_water):
        print('Finding water in plot area...')
        water_here_gdf = geopandas.overlay(water_gdf, plot_gdf, how='intersection')
    if (show_roads):
        print('Finding roads in plot area...')
        roads_here_gdf = geopandas.overlay(roads_gdf, plot_gdf, how='intersection')
    
    print('\n')
    print('Finding tracts that changed between 2019 and 2020...')
    tracts_2019_gdf = city_tracts_years_gdf.xs(2019, level='year')
    tracts_2020_gdf = city_tracts_years_gdf.xs(2020, level='year')


    tracts_2019_but_not_2020 = tracts_2019_gdf[~tracts_2019_gdf.index.isin(tracts_2020_gdf.index)]['NAME'].tolist()
    print('tracts yeeted: {0:,.0f}'.format(len(tracts_2019_but_not_2020)))
    tracts_2020_but_not_2019 = tracts_2020_gdf[~tracts_2020_gdf.index.isin(tracts_2019_gdf.index)]['NAME'].tolist()
    print('tracts gained: {0:,.0f}'.format(len(tracts_2020_but_not_2019)))

    print('\n')
    print('YEETED TRACTS:')
    print(sorted([np.float(x) for x in tracts_2019_but_not_2020]))
    print('----------')
    print('ADDED TRACTS:')
    print(sorted([np.float(x) for x in tracts_2020_but_not_2019]))
    print('\n')
    
    
    print('Plotting tracts that changed between 2019 and 2020...')
    fig, ax = plt.subplots(1,1, figsize=(18,18))

#     tracts_2019_gdf.plot(ax=ax, color='none', edgecolor='black')
#     tracts_2019_gdf[(tracts_2019_gdf['NAME'].isin(tracts_2019_but_not_2020)) 
# #                     & 
# #                     (
# #                         (pandas.to_numeric(tracts_2019_gdf['NAME'], errors='coerce', downcast='float') >= 1200)
# #                            & 
# #                         (pandas.to_numeric(tracts_2019_gdf['NAME'], errors='coerce', downcast='float') <= 1300)
# #                      )
#                 ].plot(ax=ax, color='none', edgecolor='black', linewidth=4)
#     for ix, thisrow in tracts_2019_gdf[tracts_2019_gdf.geometry.apply(lambda x: x.intersects(plot_geo))].iterrows():
#        if (thisrow['NAME'] in tracts_2019_but_not_2020):
#             if ((np.float(thisrow['NAME']) == 3103) | (np.float(thisrow['NAME']) == 5653)):
#                 annotator = '{0:}'.format(np.float(thisrow['NAME']))
#                 ax.annotate(annotator, (thisrow.geometry.centroid.x, thisrow.geometry.centroid.y), (thisrow.geometry.centroid.x, thisrow.geometry.centroid.y), 
#                             color='black')#, bbox=dict(fc='white', ec='black'))

    tracts_2020_gdf.plot(ax=ax, color='none', edgecolor='red')
    tracts_2020_gdf[(tracts_2020_gdf['NAME'].isin(tracts_2020_but_not_2019))
#                    & 
#                     (
#                         (pandas.to_numeric(tracts_2020_gdf['NAME'], errors='coerce', downcast='float') >= 1200)
#                            & 
#                         (pandas.to_numeric(tracts_2020_gdf['NAME'], errors='coerce', downcast='float') <= 1300)
#                      )
                  ].plot(ax=ax, color='none', edgecolor='red', linewidth=2, alpha=0.25, linestyle='dashed')#'black', alpha=0.25)            
    for ix, thisrow in tracts_2020_gdf[tracts_2020_gdf.geometry.apply(lambda x: x.intersects(plot_geo))].iterrows():
       if (thisrow['NAME'] in tracts_2020_but_not_2019):
            if ((np.float(thisrow['NAME']) == 5629.01) | (np.float(thisrow['NAME']) == 5653)):
                annotator = '{0:}'.format(np.float(thisrow['NAME']))
                ax.annotate(annotator, (thisrow.geometry.centroid.x, thisrow.geometry.centroid.y), (thisrow.geometry.centroid.x, thisrow.geometry.centroid.y), 
                            color='red', bbox=dict(fc='white', ec='red'))

    if (show_water):
        print('\tplotting water...')
        water_here_gdf.plot(ax=ax, color='blue', alpha=0.5)

    if (show_roads):
        print('\tplotting major roads...')
        roads_here_gdf[roads_here_gdf['RTTYP'] == 'I'].plot(ax=ax, color='purple', linewidth=5)
        roads_here_gdf[roads_here_gdf['RTTYP'] == 'U'].plot(ax=ax, color='purple', linewidth=3)
        roads_here_gdf[roads_here_gdf['RTTYP'] == 'S'].plot(ax=ax, color='purple', linewidth=2)
        print('\tplotting minor roads...')
        roads_here_gdf[roads_here_gdf['RTTYP'] == 'M'].plot(ax=ax, color='purple', linewidth=1)
        roads_here_gdf[roads_here_gdf['RTTYP'].isnull()].plot(ax=ax, color='purple', linewidth=0.5)

    ax.set_xlim(xlim)
    ax.set_ylim(ylim)
    plt.title('BLACK = OLD (2019); RED = NEW (2020)', fontsize=18)

    plt.show()
print('Done!')



How many tracts per year?
	2012: found 137 tracts!
	2013: found 137 tracts!
	2014: found 137 tracts!
	2015: found 137 tracts!
	2016: found 137 tracts!
	2017: found 137 tracts!
	2018: found 137 tracts!
	2019: found 137 tracts!
	2020: found 128 tracts!
	2021: found 128 tracts!
	2022: found 128 tracts!
Done!


In [6]:


# BALTIMORE 2019 -> 2020

# MULTIPLE 2019 TRACTS COMBINED INTO SINGLE 2020 TRACT
## Tracts 1801 and 1802 were combined into 2806

# -----------------------------------------------------------------

# WASHINGTON 2019 -> 2020

# 2019 TRACTS SPLIT INTO MULTIPLE 2020 TRACTS
### Tract 1.0 split into 1.01 and 1.02
### Tract 7.01 split into 7.03 and 7.04
### Tract 8.01 split into 8.03 and 8.04
### Tract 9.01 split into 9.03 and 9.04
### Tract 10.01 split into 10.03 and 10.04
### Tract 13.02 split into 13.03 and 13.04
### Tract 25.02 split into 25.03 and 25.04
### Tract 27.01 split into 27.03 and 27.04
### Tract 37.0 split into 37.01 and 37.02
### Tract 38.0 split into 38.01 and 38.02
### Tract 39.0 split into 39.01 and 39.02
### Tract 44.0 split into 44.01 and 44.02
### Tract 47.01 split into 47.03 and 47.04
### Tract 50.02 split into 50.03 and 50.04
### Tract 52.01 split into 52.02 and 52.03
### Tract 53.01 split into 53.02 and 53.03
### Tract 55.0 split into 55.01 and 55.02
### Tract 56.0 split into 56.01 and 56.02
### Tract 58.0 split into 58.01 and 58.02
### Tract 72.0 split into 72.01 and 72.02 and 72.03
### Tract 95.01 split into 95.10 and 95.11
### Tract 102.0 split into 102.01 and 102.02
### Tract 106.0 split into 106.01 and 106.02 and 106.03
### Tract 110.0 split into 110.01 and 110.02

# 2019 TRACTS RENAMED IN 2020
### Tract 62.01 renamed to 9800

# MINOR THINGS TO IGNORE
### Tract 39.02 includes a small uninhabited part of 27.02

# -----------------------------------------------------------------

# DETROIT 2019 -> 2020

# TRACTS NEWLY CREATED IN 2020
### Tract 9841 was created for a zero-population area (docks/water)
### Tract 9842 was created for a zero-population area (Detroit Iron & Metal)
### Tract 9858 was created for a zero-population area (Rouge Park)

# 2019 TRACTS RENAMED WITH CHANGED BORDERS FOR 2020
### Tract 5240.0 became 5240.01 after zero-pop tract created (9842)
### Tract 5241.0 became 5241.01 after zero-pop tract created (9841)
### Tract 5462.0 became 5462.01 after Rouge Park became a zero-pop tract (9858)
### Tract 5463.0 became 5463.01 after Rouge Park became a zero-pop tract (9858)

# 2019 TRACTS SPLIT INTO MULTIPLE 2020 TRACTS
### Tract 5315.0 split into 5315.01 and 5315.02

# MULTIPLE 209 TRACTS COMBINED INTO SINGLE 2020 TRACT
### Tracts 5047.0 and 5048.0 and 5055.0 combined into 5057.0
### Tracts 5049.0 and 5050.0 combined into 5056.0 
### Tracts 5078.0 and 5079.0 combined into 5090.0
### Tracts 5104.0 and 5105.0 combined into 5091.0
### Tracts 5107.0 and 5110.0 combined into 5058.0
### Tracts 5121.0 and 5122.0 combined into 5026.0
### Tracts 5123.0 and 5124.0 combined into 5027.0
### Tracts 5126.0 and 5129.0 combined into 5128.0
### Tracts 5136.0 and 5156.0 combined into 5138.0
### Tracts 5159.0 and 5161.0 combined into 5192.0
### Tracts 5162.0 and 5163.0 combined into 5193.0
### Tracts 5184.0 and 5185.0 combined into 5191.0
### Tracts 5186.0 and 5188.0 combined into 5190.0
### Tracts 5213.0 and 5221.0 combined into 5228.0 
### Tracts 5222.0 and 5272.0 and 5273.0 combined into 5279.0 
### Tracts 5249.0 and 5250.0 combined into 5246.0 (but subratcting zero-pop 9841.0)
### Tracts 5255.0 and 5256.0 combined into 5259.0 
### Tracts 5322.0 and 5323.0 combined into 5321.0 
### Tracts 5335.0 and 5337.0 combined into 5338.0 
### Tracts 5345.0 and 5346.0 combined into 5348.0 
### Tracts 5354.0 and 5355.0 combined into 5358.0 
### Tracts 5427.0 and 5428.0 combined into 5433.0 
### Tracts 5436.0 and 5438.0 combined into 5448.0 
### Tracts 5453.0 and 5454.0 combined into 5472.0 
### Tracts 5464.0 and 5465.0 combined into 5470.0 
### Tracts 5468.0 and 5469.0 combined into 5471.0 

# MINOR THINGS TO IGNORE
### Tract 5279.0 no longer includes Ken-Mac Metals plant that was in 5273.0
### Holy Cross Cemetery: North part of what was 5241.0 now in 5260.0

# -----------------------------------------------------------------

# CLEVELAND 2019 -> 2020

# TRACTS NEWLY CREATED IN 2020
### Tract 9802 was created for a zero-population area (Edgwater Park)
### Tract 9809 was created for a zero-population area (CSX railyard)

# TRACTS RENAMED FROM 2019 TO 2020
### Tract 1012.0 renamed 1012.01 after carving out zero-pop 9802    
    
# 2019 TRACTS SPLIT INTO MULTIPLE 2020 TRACTS
### Tract 1172.02 split into 1172.03 and 9809 (CSX railyard)

# MULTIPLE 2019 TRACTS COMBINED INTO SINGLE 2020 TRACT
### Tracts 1031.0 and 1034.0 combined into 1975.0
### Tracts 1039.0 and 1041.0 combined into 1977.0
### Tracts 1042.0 and 1043.0 combined into 1978.0
### Tracts 1046.0 and 1049.0 combined into 1976.0
### Tracts 1063.0 and 1064.0 combined into 1974.0
### Tracts 1105.01 and 1108.01 combined into 1979.0
### Tracts 1115.0 and 1116.0 combined into 1989.0
### Tracts 1118.0 and 1119.02 combined into 1990.0
### Tracts 1124.0 and 1128.0 combined into 1987.0
### Tracts 1125.0 and 1126.0 combined into 1988.0
### Tracts 1131.01 and 1965.0 combined into 1972.0
### Tracts 1135.0 and 1136.0 combined into 1985.0
### Tracts 1138.01 and 1141.0 combined into 1984.0
### Tracts 1143.0 and 1147.0 combined into 1148.0
### Tracts 1149.0 and 1153.0 combined into 1980.0
### Tracts 1151.0 and 1152.0 combined into 1981.0
### Tracts 1161.00 and 1162.00 were combined into 1992.00
### Tracts 1184.0 and 1185.0 combined into 1991.0
### Tracts 1187.0 and 1191.0 combined into 1968.0
### Tracts 1192.02 and 1193.0 combined into 1986.0



# -----------------------------------------------------------------

# NEWARK 2019 -> 2020

# 2019 TRACTS SPLIT INTO MULTIPLE 2020 TRACTS
### Tract 22.01 split into 22.03 and 22.04

# -----------------------------------------------------------------

# ST. LOUIS 2019 -> 2020
# MULTIPLE 2019 TRACTS COMBINED INTO SINGLE 2020 TRACT
### Tracts 1114.0 and 1115.0 combined into 1277.0
### Tracts 1184.0 and 1211.0 combined into 1278.0


# -----------------------------------------------------------------

# RICHMOND 2019 -> 2020

# 2019 TRACTS SPLIT INTO MULTIPLE 2020 TRACTS
### Tract 102 split into 102.01 and 102.02
### Tract 205 split into 205.01 and 205.02
### Tract 305 split into 305.01 and 305.02
### Tract 402 split into 402.01 and 402.02
### Tract 605 split into 605.01 and 605.02
### Tract 610 split into 610.01 and 610.02
### Tract 708.01 split into 708.03 and 708.04
### Tract 709 split into 709.01 and 709.02
### Tract 710.01 split into 710.03 and 710.04



# SAN FRANCISCO 2019 -> 2020


# NEW TRACTS
### 179.03 created for Treasure Island and Yerba Buena Island

# RENAMED TRACTS
### 179.02 renamed 9902.0 (zero-pop SF Bay)

# 2019 TRACTS SPLIT INTO MULTIPLE 2020 TRACTS
### 101.0 split into 101.01 and 101.02
### 102.0 split into 102.01 and 102.02
### 104.0 split into 104.01 and 104.02
### 107.0 split into 107.01 and 107.02
### 109.0 split into 109.01 and 109.02
### 110.0 split into 110.01 and 110.02
### 111.0 split into 111.01 and 111.02
### 120.0 split into 120.01 and 120.02
### 122.01 split into 122.03 and 122.04
### 124.01 split into 124.03 and 124.04
### 124.02 split into 124.05 and 124.06
### 125.01 split into 125.03 and 124.04
### 128.0 split into 128.01 and 128.02
### 130.0 split into 130.01 and 130.02
### 134.0 split into 134.01 and 134.02
### 152.0 split into 152.01 and 152.02
### 154.0 split into 154.01 and 154.02
### 157.0 split into 157.01 and 157.02
### 161.0 split into 161.01 and 161.02
### 166.0 split into 166.01 and 166.02
### 176.01 split into 176.02 and 176.03 and 176.04
### 178.02 split into 178.03 and 178.04
### 201.0 split into 201.01 and 201.02
### 202.0 split into 202.01 and 202.02
### 206.0 split into 206.01 and 206.02
### 207.0 split into 207.01 and 207.02
### 208.0 split into 208.01 and 208.02
### 255.0 split into 255.01 and 255.02
### 262.0 split into 262.01 and 262.02
### 314.0 split into 314.01 and 314.02
### 330.0 split into 330.01 and 330.02
### 351.0 split into 351.01 and 351.02
### 452.0 split into 452.01 and 452.02
### 479.01 split into 479.03 and 479.04
### 607.0 split into 607.01 and 607.02 and 607.03
### 611.0 split into 611.01 and 611.02
### 614.0 split into 614.01 and 614.02
### 615.0 split into 615.01 and 615.02 and 615.03 and 615.04 and 615.05 and 615.06 and 615.07 and 615.08

# MINOR THINGS TO IGNORE
# Yacht berths by Marina Airfield moved from 126.02 to 127


# -----------------------------------------------------------------

# PITTSBURGH 2019 -> 2020

# SPLIT
# 103.0 split into 103.01 and 103.02
# 5629.0 split into 5629.01 + 5653
# 5632.0 split into 5632.01 and 5632.02

# COMBINED
# 1016 and 1017 combined into 1019
# 1204 and 1208 combined into 1209
# 1207 and 1301 combined into 1307
# 1303 and 1304 combined into 1308
# 1406 and 1410 combined into 1412
# 2107 and 2507 combined into 5652
# 2206 and 2503 combined into 5651
# 2406 and 2412 combined into 2413
# 2609 and 2612 combined into 2613
# 2704 and 2715 combined into 2716
# 5616.0 and 5617.0 combined into 1610




In [37]:
s = time.time() 
tract_changes = {}
tract_changes['Baltimore'] = {}
tract_changes['Baltimore']['combined'] = []
tract_changes['Baltimore']['combined'].append({2019: ['180100', '180200'], 2020: ['280600']})

# -----------------------------------------------------------------


tract_changes['Washington DC'] = {}
tract_changes['Washington DC']['renamed'] = []
tract_changes['Washington DC']['renamed'].append({2019: ['006200'], 2020: ['980000']})

tract_changes['Washington DC']['split'] = []
tract_changes['Washington DC']['split'].append({2019: ['000100'], 2020: ['000101', '000102']})
tract_changes['Washington DC']['split'].append({2019: ['000701'], 2020: ['000703', '000704']})
tract_changes['Washington DC']['split'].append({2019: ['000801'], 2020: ['000803', '000804']})
tract_changes['Washington DC']['split'].append({2019: ['001001'], 2020: ['001003', '001004']})
tract_changes['Washington DC']['split'].append({2019: ['001302'], 2020: ['001303', '001304']})
tract_changes['Washington DC']['split'].append({2019: ['002502'], 2020: ['002503', '002504']})
tract_changes['Washington DC']['split'].append({2019: ['002701'], 2020: ['002703', '002704']})
tract_changes['Washington DC']['split'].append({2019: ['003700'], 2020: ['003701', '003702']})
tract_changes['Washington DC']['split'].append({2019: ['003800'], 2020: ['003801', '003802']})
tract_changes['Washington DC']['split'].append({2019: ['003900'], 2020: ['003901', '003902']})
tract_changes['Washington DC']['split'].append({2019: ['004400'], 2020: ['004401', '004402']})
tract_changes['Washington DC']['split'].append({2019: ['004701'], 2020: ['004703', '004704']})
tract_changes['Washington DC']['split'].append({2019: ['005002'], 2020: ['005003', '005004']})
tract_changes['Washington DC']['split'].append({2019: ['005201'], 2020: ['005202', '005203']})
tract_changes['Washington DC']['split'].append({2019: ['005301'], 2020: ['005302', '005303']})
tract_changes['Washington DC']['split'].append({2019: ['005500'], 2020: ['005501', '005502']})
tract_changes['Washington DC']['split'].append({2019: ['005600'], 2020: ['005601', '005602']})
tract_changes['Washington DC']['split'].append({2019: ['005800'], 2020: ['005801', '005802']})
tract_changes['Washington DC']['split'].append({2019: ['007200'], 2020: ['007201', '007202']})
tract_changes['Washington DC']['split'].append({2019: ['009501'], 2020: ['009510', '009511']})
tract_changes['Washington DC']['split'].append({2019: ['010200'], 2020: ['010201', '010202']})
tract_changes['Washington DC']['split'].append({2019: ['010600'], 2020: ['010601', '010602']})
tract_changes['Washington DC']['split'].append({2019: ['011000'], 2020: ['011001', '011002']})


# -----------------------------------------------------------------

tract_changes['Detroit'] = {}
tract_changes['Detroit']['new'] = []
tract_changes['Detroit']['new'].append({2019: [''], 2020: ['984100']})
tract_changes['Detroit']['new'].append({2019: [''], 2020: ['984200']})
tract_changes['Detroit']['new'].append({2019: [''], 2020: ['985800']})

tract_changes['Detroit']['renamed'] = []
tract_changes['Detroit']['renamed'].append({2019: ['524000'], 2020: ['524001']})
tract_changes['Detroit']['renamed'].append({2019: ['524100'], 2020: ['524101']})
tract_changes['Detroit']['renamed'].append({2019: ['546200'], 2020: ['546201']})
tract_changes['Detroit']['renamed'].append({2019: ['546300'], 2020: ['546301']})

tract_changes['Detroit']['split'] = []
tract_changes['Detroit']['split'].append({2019: ['531500'], 2020: ['531501', '531502']})

tract_changes['Detroit']['combined'] = []
tract_changes['Detroit']['combined'].append({2019: ['504700', '5048700'], 2020: ['505700']})
tract_changes['Detroit']['combined'].append({2019: ['504900', '505000'], 2020: ['505600']})
tract_changes['Detroit']['combined'].append({2019: ['507800', '507900'], 2020: ['509000']})
tract_changes['Detroit']['combined'].append({2019: ['510400', '510500'], 2020: ['509100']})
tract_changes['Detroit']['combined'].append({2019: ['510700', '511100'], 2020: ['510800']})
tract_changes['Detroit']['combined'].append({2019: ['511200', '512100'], 2020: ['502600']})
tract_changes['Detroit']['combined'].append({2019: ['512300', '512400'], 2020: ['502700']})
tract_changes['Detroit']['combined'].append({2019: ['512600', '512900'], 2020: ['512800']})
tract_changes['Detroit']['combined'].append({2019: ['513600', '515600'], 2020: ['513800']})
tract_changes['Detroit']['combined'].append({2019: ['515900', '516100'], 2020: ['519200']})
tract_changes['Detroit']['combined'].append({2019: ['516200', '516300'], 2020: ['519300']})
tract_changes['Detroit']['combined'].append({2019: ['518400', '518500'], 2020: ['519100']})
tract_changes['Detroit']['combined'].append({2019: ['518600', '518800'], 2020: ['519000']})
tract_changes['Detroit']['combined'].append({2019: ['521300', '522100'], 2020: ['522800']})
tract_changes['Detroit']['combined'].append({2019: ['522200', '527200', '527300'], 2020: ['527900']})
tract_changes['Detroit']['combined'].append({2019: ['524900', '525000'], 2020: ['524600']})
tract_changes['Detroit']['combined'].append({2019: ['525500', '525600'], 2020: ['525900']})
tract_changes['Detroit']['combined'].append({2019: ['532200', '532300'], 2020: ['532100']})
tract_changes['Detroit']['combined'].append({2019: ['533500', '533700'], 2020: ['533800']})
tract_changes['Detroit']['combined'].append({2019: ['534500', '534600'], 2020: ['534800']})
tract_changes['Detroit']['combined'].append({2019: ['542700', '542800'], 2020: ['543300']})
tract_changes['Detroit']['combined'].append({2019: ['543600', '543800'], 2020: ['544800']})
tract_changes['Detroit']['combined'].append({2019: ['545300', '545400'], 2020: ['547200']})
tract_changes['Detroit']['combined'].append({2019: ['546400', '546500'], 2020: ['547000']})
tract_changes['Detroit']['combined'].append({2019: ['546800', '546900'], 2020: ['547100']})

# -----------------------------------------------------------------

tract_changes['Cleveland'] = {}
tract_changes['Cleveland']['new'] = []
tract_changes['Cleveland']['new'].append({2019: [''], 2020: ['980200']})
tract_changes['Cleveland']['new'].append({2019: [''], 2020: ['980900']})

tract_changes['Cleveland']['renamed'] = []
tract_changes['Cleveland']['renamed'].append({2019: ['101200'], 2020: ['101201']})

tract_changes['Cleveland']['split'] = []
tract_changes['Cleveland']['split'].append({2019: ['117202'], 2020: ['117203', '980900']})

tract_changes['Cleveland']['combined'] = []
tract_changes['Cleveland']['combined'].append({2019: ['103100', '103400'], 2020: ['197500']})
tract_changes['Cleveland']['combined'].append({2019: ['103900', '104100'], 2020: ['197700']})
tract_changes['Cleveland']['combined'].append({2019: ['104200', '104300'], 2020: ['197800']})
tract_changes['Cleveland']['combined'].append({2019: ['104600', '104900'], 2020: ['197600']})
tract_changes['Cleveland']['combined'].append({2019: ['106300', '106400'], 2020: ['197400']})
tract_changes['Cleveland']['combined'].append({2019: ['110501', '110801'], 2020: ['197900']})
tract_changes['Cleveland']['combined'].append({2019: ['111500', '111600'], 2020: ['198900']})
tract_changes['Cleveland']['combined'].append({2019: ['111800', '111902'], 2020: ['199000']})
tract_changes['Cleveland']['combined'].append({2019: ['112400', '112800'], 2020: ['198700']})
tract_changes['Cleveland']['combined'].append({2019: ['112500', '112600'], 2020: ['198800']})
tract_changes['Cleveland']['combined'].append({2019: ['113101', '196500'], 2020: ['197200']})
tract_changes['Cleveland']['combined'].append({2019: ['113500', '113600'], 2020: ['198500']})
tract_changes['Cleveland']['combined'].append({2019: ['113801', '114100'], 2020: ['198400']})
tract_changes['Cleveland']['combined'].append({2019: ['114300', '114700'], 2020: ['114800']})
tract_changes['Cleveland']['combined'].append({2019: ['114900', '115300'], 2020: ['198000']})
tract_changes['Cleveland']['combined'].append({2019: ['115100', '115200'], 2020: ['198100']})
tract_changes['Cleveland']['combined'].append({2019: ['116100', '116200'], 2020: ['199200']})
tract_changes['Cleveland']['combined'].append({2019: ['118400', '118500'], 2020: ['199100']})
tract_changes['Cleveland']['combined'].append({2019: ['118700', '119100'], 2020: ['196800']})
tract_changes['Cleveland']['combined'].append({2019: ['119202', '119300'], 2020: ['198600']})


# -----------------------------------------------------------------

tract_changes['Newark'] = {}
tract_changes['Newark']['split'] = []
tract_changes['Newark']['split'].append({2019: ['002201'], 2020: ['002203', '002204']})


# -----------------------------------------------------------------

tract_changes['St. Louis'] = {}
tract_changes['St. Louis']['combined'] = []
tract_changes['St. Louis']['combined'].append({2019: ['111400', '111500'], 2020: ['127700']})
tract_changes['St. Louis']['combined'].append({2019: ['118400', '121100'], 2020: ['127800']})

# -----------------------------------------------------------------

tract_changes['Richmond'] = {}
tract_changes['Richmond']['split'] = []
tract_changes['Richmond']['split'].append({2019: ['010200'], 2020: ['010201', '010202']})
tract_changes['Richmond']['split'].append({2019: ['020500'], 2020: ['020501', '020502']})
tract_changes['Richmond']['split'].append({2019: ['030500'], 2020: ['030501', '030502']})
tract_changes['Richmond']['split'].append({2019: ['040200'], 2020: ['040201', '040202']})
tract_changes['Richmond']['split'].append({2019: ['060500'], 2020: ['060501', '060502']})
tract_changes['Richmond']['split'].append({2019: ['061000'], 2020: ['061001', '061002']})
tract_changes['Richmond']['split'].append({2019: ['070801'], 2020: ['070803', '070804']})
tract_changes['Richmond']['split'].append({2019: ['070900'], 2020: ['070901', '070902']})
tract_changes['Richmond']['split'].append({2019: ['071001'], 2020: ['071003', '071004']})

# -----------------------------------------------------------------

tract_changes['San Francisco'] = {}
tract_changes['San Francisco']['new'] = []
tract_changes['San Francisco']['new'].append({2019: [''], 2020: ['017903']})

tract_changes['San Francisco']['renamed'] = []
tract_changes['San Francisco']['renamed'].append({2019: ['017903'], 2020: ['990200']})

tract_changes['San Francisco']['split'] = []
tract_changes['San Francisco']['split'].append({2019: ['010100'], 2020: ['010101', '010102']})
tract_changes['San Francisco']['split'].append({2019: ['010200'], 2020: ['010201', '010202']})
tract_changes['San Francisco']['split'].append({2019: ['010400'], 2020: ['010401', '010402']})
tract_changes['San Francisco']['split'].append({2019: ['010700'], 2020: ['010701', '010702']})
tract_changes['San Francisco']['split'].append({2019: ['010900'], 2020: ['010901', '010902']})
tract_changes['San Francisco']['split'].append({2019: ['011000'], 2020: ['011001', '011002']})
tract_changes['San Francisco']['split'].append({2019: ['011100'], 2020: ['011101', '011102']})
tract_changes['San Francisco']['split'].append({2019: ['012000'], 2020: ['012001', '012002']})
tract_changes['San Francisco']['split'].append({2019: ['012201'], 2020: ['012203', '012204']})
tract_changes['San Francisco']['split'].append({2019: ['012401'], 2020: ['012403', '012404']})
tract_changes['San Francisco']['split'].append({2019: ['012402'], 2020: ['012405', '012406']})
tract_changes['San Francisco']['split'].append({2019: ['012501'], 2020: ['012503', '012504']})
tract_changes['San Francisco']['split'].append({2019: ['012800'], 2020: ['012801', '012802']})
tract_changes['San Francisco']['split'].append({2019: ['013000'], 2020: ['013001', '013002']})
tract_changes['San Francisco']['split'].append({2019: ['013000'], 2020: ['013401', '013402']})
tract_changes['San Francisco']['split'].append({2019: ['015200'], 2020: ['015201', '015202']})
tract_changes['San Francisco']['split'].append({2019: ['015700'], 2020: ['015701', '015702']})
tract_changes['San Francisco']['split'].append({2019: ['016100'], 2020: ['016100', '016100']})
tract_changes['San Francisco']['split'].append({2019: ['016600'], 2020: ['016600', '016600']})
tract_changes['San Francisco']['split'].append({2019: ['017601'], 2020: ['017602', '017603', '017604']})
tract_changes['San Francisco']['split'].append({2019: ['017801'], 2020: ['017802', '017803']})
tract_changes['San Francisco']['split'].append({2019: ['020100'], 2020: ['020101', '020102']})
tract_changes['San Francisco']['split'].append({2019: ['020200'], 2020: ['020201', '020202']})
tract_changes['San Francisco']['split'].append({2019: ['020600'], 2020: ['020601', '020602']})
tract_changes['San Francisco']['split'].append({2019: ['020700'], 2020: ['020701', '020702']})
tract_changes['San Francisco']['split'].append({2019: ['020800'], 2020: ['020801', '020802']})
tract_changes['San Francisco']['split'].append({2019: ['025500'], 2020: ['025501', '025502']})
tract_changes['San Francisco']['split'].append({2019: ['026200'], 2020: ['026201', '026202']})
tract_changes['San Francisco']['split'].append({2019: ['031400'], 2020: ['031401', '031402']})
tract_changes['San Francisco']['split'].append({2019: ['033000'], 2020: ['033001', '033002']})
tract_changes['San Francisco']['split'].append({2019: ['035100'], 2020: ['035101', '035102']})
tract_changes['San Francisco']['split'].append({2019: ['045200'], 2020: ['045201', '045202']})
tract_changes['San Francisco']['split'].append({2019: ['047901'], 2020: ['047903', '047904']})
tract_changes['San Francisco']['split'].append({2019: ['060700'], 2020: ['060701', '060702', '060703']})
tract_changes['San Francisco']['split'].append({2019: ['061100'], 2020: ['061101', '061102']})
tract_changes['San Francisco']['split'].append({2019: ['061100'], 2020: ['061401', '061402']})
tract_changes['San Francisco']['split'].append({2019: ['061500'], 2020: ['061501', '061502', '061503', '061504', '061505', '061506', '061507', '061508']})

# -----------------------------------------------------------------

tract_changes['Philadelphia'] = {}

tract_changes['Philadelphia']['new'] = []
tract_changes['Philadelphia']['new'].append({2019: '', 2020: ['989300']})

tract_changes['Philadelphia']['renamed'] = []
tract_changes['Philadelphia']['renamed'].append({2019: ['005000'], 2020: ['989200']})
tract_changes['Philadelphia']['renamed'].append({2019: ['006900'], 2020: ['980900']})

tract_changes['Philadelphia']['split'] = []
tract_changes['Philadelphia']['split'].append({2019: ['000100'], 2020: ['000101', '000102']})
tract_changes['Philadelphia']['split'].append({2019: ['000402'], 2020: ['000403', '000404']})
tract_changes['Philadelphia']['split'].append({2019: ['000700'], 2020: ['000701', '000702']})
tract_changes['Philadelphia']['split'].append({2019: ['000804'], 2020: ['000805', '000806']})
tract_changes['Philadelphia']['split'].append({2019: ['001202'], 2020: ['001203', '001204']})
tract_changes['Philadelphia']['split'].append({2019: ['001300'], 2020: ['001301', '001302']})
tract_changes['Philadelphia']['split'].append({2019: ['004102'], 2020: ['004103', '004104']})
tract_changes['Philadelphia']['split'].append({2019: ['012500'], 2020: ['012501', '012502']})
tract_changes['Philadelphia']['split'].append({2019: ['013700'], 2020: ['013701', '013702']})
tract_changes['Philadelphia']['split'].append({2019: ['014200'], 2020: ['014201', '014202']})
tract_changes['Philadelphia']['split'].append({2019: ['016000'], 2020: ['016001', '016002']})
tract_changes['Philadelphia']['split'].append({2019: ['018800'], 2020: ['018801', '018802']})
tract_changes['Philadelphia']['split'].append({2019: ['020700'], 2020: ['020701', '020702']})
tract_changes['Philadelphia']['split'].append({2019: ['036900'], 2020: ['036901', '036902']})
tract_changes['Philadelphia']['split'].append({2019: ['038300'], 2020: ['038301', '038302']})
tract_changes['Philadelphia']['split'].append({2019: ['039000'], 2020: ['039001', '039002']})
tract_changes['Philadelphia']['split'].append({2019: ['980000'], 2020: ['980001', '980002', '980003']})
tract_changes['Philadelphia']['split'].append({2019: ['980700'], 2020: ['980701', '980702']})
tract_changes['Philadelphia']['split'].append({2019: ['980900'], 2020: ['980901', '980902', '980903', '980904', '980905', '980906']})

# -----------------------------------------------------------------

tract_changes['Pittsburgh'] = {}
tract_changes['Pittsburgh']['split'] = []
tract_changes['Pittsburgh']['split'].append({2019: ['010300'], 2020: ['010301', '010302']})
tract_changes['Pittsburgh']['split'].append({2019: ['562900'], 2020: ['562901', '565300']})
tract_changes['Pittsburgh']['split'].append({2019: ['563200'], 2020: ['563201', '563202']})

tract_changes['Pittsburgh']['combined'] = []
tract_changes['Pittsburgh']['combined'].append({2019: ['101600', '101700'], 2020: ['101900']})
tract_changes['Pittsburgh']['combined'].append({2019: ['120400', '120800'], 2020: ['120900']})
tract_changes['Pittsburgh']['combined'].append({2019: ['120700', '130100'], 2020: ['130700']})
tract_changes['Pittsburgh']['combined'].append({2019: ['130300', '130400'], 2020: ['130800']})
tract_changes['Pittsburgh']['combined'].append({2019: ['140600', '141000'], 2020: ['141200']})
tract_changes['Pittsburgh']['combined'].append({2019: ['210700', '250700'], 2020: ['565200']})
tract_changes['Pittsburgh']['combined'].append({2019: ['220600', '250300'], 2020: ['565100']})
tract_changes['Pittsburgh']['combined'].append({2019: ['260900', '261200'], 2020: ['261300']})
tract_changes['Pittsburgh']['combined'].append({2019: ['270400', '271500'], 2020: ['271600']})
tract_changes['Pittsburgh']['combined'].append({2019: ['561600', '561700'], 2020: ['161000']})

#pprint(tract_changes)
with open(output_data_dir+"changes_to_tracts.json", "w") as outfile: 
    json.dump(tract_changes, outfile)
    
with open(output_data_dir+"changes_to_tracts.json", "r") as readthefile: 
    y = json.load(readthefile)

e = time.time()
g += e-s
print('File written!')


File written!


# Get geo-aggregated loans for this city

and join the tract data onto the loans data


In [8]:
s = time.time()

print('reading nationwide data...')
agg_loans_all_df = pandas.read_csv(loans_dir+'agg_loans.csv', encoding='utf-8', low_memory=False, index_col='rownumber', keep_default_na=False)
agg_loans_all_df = agg_loans_all_df.rename(columns={'activity_year': 'year'})
e = time.time()
print('Read {0:,.0f} nationwide tract-years in {1:,.0f} seconds...'.format(len(agg_loans_all_df), e-s))

# Keep only business loans
agg_loans_all_df = agg_loans_all_df[agg_loans_all_df['loan_type'] == 4]
# Keep only loan originations
agg_loans_all_df = agg_loans_all_df[agg_loans_all_df['action_taken_type'] == 1]
# Keep only this state
agg_loans_all_df = agg_loans_all_df[agg_loans_all_df['state'] == thestate]


# Construct all GEOIDs
agg_loans_all_df = agg_loans_all_df.assign(GEOID = np.nan)

agg_loans_all_df.loc[agg_loans_all_df['census_tract'].apply(lambda x: np.floor(x) == x), 'GEOID'] = agg_loans_all_df[
    agg_loans_all_df['census_tract'].apply(lambda x: np.floor(x) == x)
].apply(lambda row: '14000US{0:02d}{1:03d}{2:04d}{3:02d}'.format(
    row['state'], row['county'], 
    int(np.floor(row['census_tract'])),
    int(np.round(row['census_tract'] - np.floor(row['census_tract']),0)),
    row['census_tract']
), axis=1)#[['state','county','census_tract']]#.drop_duplicates().sort_values(by='census_tract')

agg_loans_all_df.loc[agg_loans_all_df['census_tract'].apply(lambda x: np.floor(10*x) != 10*x), 'GEOID'] = agg_loans_all_df[
    agg_loans_all_df['census_tract'].apply(lambda x: np.floor(10*x) != 10*x)
].apply(lambda row: '14000US{0:02d}{1:03d}{2:04d}{3:02d}'.format(
    row['state'], row['county'], 
    int(np.floor(row['census_tract'])),
    int(np.round(100*(row['census_tract'] - np.floor(row['census_tract'])),0))
), axis=1)

geoids_so_far_list = agg_loans_all_df['GEOID'].dropna().drop_duplicates().sort_values()

agg_loans_all_df.loc[(agg_loans_all_df['GEOID'].isnull()), 'GEOID'] = agg_loans_all_df[
    (agg_loans_all_df['GEOID'].isnull()) 
].apply(lambda row: 
        '14000US{0:02d}{1:03d}{2:04d}{3:02d}'.format(
            row['state'], row['county'], 
            int(np.floor(row['census_tract'])),
            int(np.round(100*(row['census_tract'] - np.floor(row['census_tract'])),0))
            ), axis=1
       )

# Done building GEOIDs! Now let's join to city tracts we found earlier
agg_loans_all_df = agg_loans_all_df.set_index(['GEOID','year'])

data_gdf = city_tracts_years_gdf.join(agg_loans_all_df[[x for x in agg_loans_all_df.columns if x not in city_tracts_years_gdf.columns]], how='left')



print('converting columns to numeric...')
numeric_columns = []
numeric_columns += ['nLoans1', 'amtLoans1', 'nLoans100k', 'amtLoans100k']
numeric_columns += ['nLoans250k', 'amtLoans250k', 'nLoansToSmallest', 'amtLoansToSmallest']

for x in numeric_columns:
    data_gdf.loc[:, x] = pandas.to_numeric(data_gdf[x], errors='coerce')

print('calculating total loans...')
data_gdf = data_gdf.assign(nLoans = data_gdf['nLoans1'] + data_gdf['nLoans100k'] + data_gdf['nLoans250k'])
data_gdf = data_gdf.assign(amtLoans = data_gdf['amtLoans1'] + data_gdf['amtLoans100k'] + data_gdf['amtLoans250k'])

# print('backing up...')
# data_gdf_bk = data_gdf

e = time.time()
g = g + (e-s)
print('\n')
print('Kept {0:,.0f} tract-years in {1:,.0f} seconds!'.format(len(data_gdf), e-s))



reading nationwide data...
Read 3,686,478 nationwide tract-years in 24 seconds...
converting columns to numeric...
calculating total loans...


Kept 1,480 tract-years in 35 seconds!


## Add income groups, CRA levels, working loans for each tract-year

In [9]:
s = time.time()

# print('getting from backup...')
# data_gdf = data_gdf_bk

print('looking up income group names from income_group_total...')

data_gdf = data_gdf.rename(columns = {'income_group_total': 'income_group_code'})
data_gdf = data_gdf.assign(income_group = np.nan)

data_gdf.loc[data_gdf['income_group_code'] == 1, 'income_group'] = '< 10% of Median Family Income (MFI)'
data_gdf.loc[data_gdf['income_group_code'] == 2, 'income_group'] = '10% to 20% of MFI'
data_gdf.loc[data_gdf['income_group_code'] == 3, 'income_group'] = '20% to 30% of MFI'
data_gdf.loc[data_gdf['income_group_code'] == 4, 'income_group'] = '30% to 40% of MFI'
data_gdf.loc[data_gdf['income_group_code'] == 5, 'income_group'] = '40% to 50% of MFI'
data_gdf.loc[data_gdf['income_group_code'] == 6, 'income_group'] = '50% to 60% of MFI'
data_gdf.loc[data_gdf['income_group_code'] == 7, 'income_group'] = '60% to 70% of MFI'
data_gdf.loc[data_gdf['income_group_code'] == 8, 'income_group'] = '70% to 80% of MFI'
data_gdf.loc[data_gdf['income_group_code'] == 9, 'income_group'] = '80% to 90% of MFI'
data_gdf.loc[data_gdf['income_group_code'] == 10, 'income_group'] = '90% to 100% of MFI'
data_gdf.loc[data_gdf['income_group_code'] == 11, 'income_group'] = '100% to 110% of MFI'
data_gdf.loc[data_gdf['income_group_code'] == 12, 'income_group'] = '110% to 120% of MFI'
data_gdf.loc[data_gdf['income_group_code'] == 13, 'income_group'] = '> 120% of MFI'
data_gdf.loc[data_gdf['income_group_code'] == 14, 'income_group'] = 'unknown'

print('Adding CRA income levels (low/moderate/middle/upper/unknown)...')
# Get levels (low, moderate, middle, upper)
data_gdf = data_gdf.assign(cra_level = np.nan)
data_gdf.loc[(data_gdf['income_group_code'] >= 1) & (data_gdf['income_group_code'] <= 5), 'cra_level'] = 'low'
data_gdf.loc[(data_gdf['income_group_code'] >= 6) & (data_gdf['income_group_code'] <= 8), 'cra_level'] = 'moderate'
data_gdf.loc[(data_gdf['income_group_code'] >= 9) & (data_gdf['income_group_code'] <= 12), 'cra_level'] = 'middle'
data_gdf.loc[(data_gdf['income_group_code'] == 13), 'cra_level'] = 'upper'
data_gdf.loc[(data_gdf['income_group_code'] == 14), 'cra_level'] = 'unknown'

print('Getting CRA income levels for tracts where only CRA level was reported...')
data_gdf.loc[data_gdf['income_group_code'] == 101, 'cra_level'] = 'low'
data_gdf.loc[data_gdf['income_group_code'] == 102, 'cra_level'] = 'moderate'
data_gdf.loc[data_gdf['income_group_code'] == 103, 'cra_level'] = 'middle'
data_gdf.loc[data_gdf['income_group_code'] == 104, 'cra_level'] = 'upper'
data_gdf.loc[data_gdf['income_group_code'] == 105, 'cra_level'] = 'unknown'

print('calculating working loans...')
data_gdf = data_gdf.assign(avgSmallLoan = data_gdf['amtLoans1'] / data_gdf['nLoans1'])

data_gdf = data_gdf.assign(nWorkingLoans = 0)
data_gdf.loc[data_gdf['avgSmallLoan'] < 10000, 
                           'nWorkingLoans'] = data_gdf['nLoans'][data_gdf['avgSmallLoan'] < 10000] - data_gdf['nLoans1'][data_gdf['avgSmallLoan'] < 10000]
data_gdf.loc[data_gdf['avgSmallLoan'] >= 10000, 
                           'nWorkingLoans'] = data_gdf['nLoans'][data_gdf['avgSmallLoan'] >= 10000]

data_gdf = data_gdf.assign(amtWorkingLoans = 0)
data_gdf.loc[data_gdf['avgSmallLoan'] < 10000, 
                           'amtWorkingLoans'] = data_gdf['amtLoans'][data_gdf['avgSmallLoan'] < 10000] - data_gdf['amtLoans1'][data_gdf['avgSmallLoan'] < 10000]
data_gdf.loc[data_gdf['avgSmallLoan'] >= 10000, 
                           'amtWorkingLoans'] = data_gdf['amtLoans'][data_gdf['avgSmallLoan'] >= 10000]

e = time.time()
g = g + (e-s)

# print('backing up...')
# data_gdf_bk = data_gdf

print('Kept {0:,.0f} tract-years in {1:} in {2:,.2f} seconds!'.format(len(data_gdf), city, e-s))



looking up income group names from income_group_total...
Adding CRA income levels (low/moderate/middle/upper/unknown)...
Getting CRA income levels for tracts where only CRA level was reported...
calculating working loans...
Kept 1,480 tract-years in Pittsburgh in 0.12 seconds!


## Connect to jobs data

### Get raw jobs table and aggregate by year

In [10]:
s = time.time()
print('getting raw jobs files...')
raw_jobs_df = pandas.DataFrame()

wac_file_list = sorted([
    '{0:}{1:}/wac/'.format(jobs_dir,state_abbrev)+x for x 
    in os.listdir('{0:}{1:}/wac'.format(jobs_dir, state_abbrev)) if 
    (x[-4:] == '.csv') and (int(x[-8:-4]) in years) and ('_S000_JT02' in x)]
)

cnt = 0
for this_file in wac_file_list:
    #if (np.mod(cnt, 50) == 0):
#    print('Reading file {0:,.0f} of {1:,.0f}...'.format(cnt, len(wac_file_list)))
    raw_jobs_df_i = pandas.read_csv(this_file)
    raw_jobs_df_i = raw_jobs_df_i.assign(job_segment = this_file[-13:-9])
    raw_jobs_df_i = raw_jobs_df_i.assign(job_type = this_file[-18:-14])
    raw_jobs_df_i = raw_jobs_df_i.assign(year = int(this_file[-8:-4]))
    raw_jobs_df = pandas.concat((raw_jobs_df, raw_jobs_df_i), axis=0)
    cnt += 1

raw_jobs_df = raw_jobs_df.reset_index(drop=True)


print('documenting job type and segment...')
job_type_metadata_list = []
job_type_metadata_list.append({'variable': 'w_geocode', 'description': 'Workplace Census Block Code'})
job_type_metadata_list.append({'variable': 'h_geocode', 'description': 'Workplace Residence Block Code'})
job_type_metadata_list.append({'variable': 'S000', 'description': 'Total number of jobs'})
job_type_metadata_list.append({'variable': 'SA01', 'description': 'Number of jobs of workers age 29 or younger'})
job_type_metadata_list.append({'variable': 'SA02', 'description': 'Number of jobs for workers age 30 to 54'})
job_type_metadata_list.append({'variable': 'SA03', 'description': 'Number of jobs for workers age 55 or older'})
job_type_metadata_list.append({'variable': 'SE01', 'description': 'Number of jobs with earnings $1250/month or less'})
job_type_metadata_list.append({'variable': 'SE02', 'description': 'Number of jobs with earnings $1251/month to $3333/month'})
job_type_metadata_list.append({'variable': 'SE03', 'description': 'Number of jobs with earnings greater than $3333/month'})
job_type_metadata_list.append({'variable': 'SI01', 'description': 'Number of jobs in Goods Producing industry sectors'})
job_type_metadata_list.append({'variable': 'SI02', 'description': "Number of jobs in Trade, Transportation, and Utilities industry sectors"})
job_type_metadata_list.append({'variable': 'SI03', 'description': 'Number of jobs in All Other Services industry sectors'})
job_type_metadata_list.append({'variable': 'createdate', 'description': "Date on which data was created, formatted as YYYYMMDD"})
job_type_metadata_df = pandas.DataFrame.from_records(job_type_metadata_list).set_index('variable')

job_segment_metadata_list = []
job_segment_metadata_list.append({'variable': 'JT00', 'description': 'All Jobs'})
job_segment_metadata_list.append({'variable': 'JT01', 'description': 'Primary Jobs'})
job_segment_metadata_list.append({'variable': 'JT02', 'description': 'All Private Jobs'})
job_segment_metadata_list.append({'variable': 'JT03', 'description': 'Private Primary Jobs'})
job_segment_metadata_list.append({'variable': 'JT04', 'description': 'All Federal Jobs'})
job_segment_metadata_list.append({'variable': 'JT05', 'description': 'Federal Primary Jobs'})
job_segment_metadata_df = pandas.DataFrame.from_records(job_segment_metadata_list).set_index('variable')


raw_jobs_df = raw_jobs_df.merge(job_type_metadata_df.reset_index(), left_on='job_type', right_on='variable').drop('variable', axis=1).rename(columns={'description': 'job_type_description'})
raw_jobs_df = raw_jobs_df.merge(job_segment_metadata_df.reset_index(), left_on='job_segment', right_on='variable').drop('variable', axis=1).rename(columns={'description': 'job_segment_description'})

print('getting geo crosswalk file...')
geo_walk_df = pandas.read_csv('{0:}{1:}/{1:}_xwalk.csv'.format(jobs_dir,state_abbrev), low_memory=False)#, index_col='tabblk2010')

print('getting GEOIDs from crosswalk file...')
raw_jobs_df = raw_jobs_df.merge(geo_walk_df[['tabblk2020','cty','trct','ctyname','trctname']], how='left', left_on='w_geocode', right_on='tabblk2020')
if (thestate <= 9):
    raw_jobs_df = raw_jobs_df.assign(GEOID = raw_jobs_df['trct'].apply(lambda x: '14000US0'+str(x)))
else:
    raw_jobs_df = raw_jobs_df.assign(GEOID = raw_jobs_df['trct'].apply(lambda x: '14000US'+str(x)))

print('selecting jobs in city...')
raw_jobs_df = raw_jobs_df[raw_jobs_df['GEOID'].isin(data_gdf.index.get_level_values('GEOID').tolist())]

print('finding columns to sum over...')
jobs_metadata_df = pandas.read_csv(code_lookup_dir+'wac_jobs_metadata.csv', encoding='utf-8', index_col='varnum')

jobs_total_columns = jobs_metadata_df[jobs_metadata_df['variable'] == 'C000']['variable'].tolist()
jobs_age_columns = jobs_metadata_df[jobs_metadata_df['variable'].apply(lambda x: x[0:2] == 'CA')]['variable'].tolist()
jobs_earnings_columns = jobs_metadata_df[jobs_metadata_df['variable'].apply(lambda x: x[0:2] == 'CE')]['variable'].tolist()
jobs_sector_columns = jobs_metadata_df[jobs_metadata_df['variable'].apply(lambda x: x[0:3] == 'CNS')]['variable'].tolist()
jobs_race_columns = jobs_metadata_df[jobs_metadata_df['variable'].apply(lambda x: x[0:2] == 'CR')]['variable'].tolist()
jobs_ethnicity_columns = jobs_metadata_df[jobs_metadata_df['variable'].apply(lambda x: x[0:2] == 'CT')]['variable'].tolist()
jobs_education_columns = jobs_metadata_df[jobs_metadata_df['variable'].apply(lambda x: x[0:2] == 'CD')]['variable'].tolist()
jobs_sex_columns = jobs_metadata_df[jobs_metadata_df['variable'].apply(lambda x: x[0:2] == 'CS')]['variable'].tolist()
jobs_firm_age_columns = jobs_metadata_df[jobs_metadata_df['variable'].apply(lambda x: x[0:3] == 'CFA')]['variable'].tolist()
jobs_firm_size_columns = jobs_metadata_df[jobs_metadata_df['variable'].apply(lambda x: x[0:3] == 'CFS')]['variable'].tolist()

jobs_columns = jobs_total_columns + jobs_age_columns + jobs_earnings_columns + jobs_sector_columns
jobs_columns += jobs_race_columns + jobs_ethnicity_columns + jobs_education_columns + jobs_sex_columns
jobs_columns += jobs_firm_age_columns + jobs_firm_size_columns

jobs_metadata_df = jobs_metadata_df.set_index('variable')


print('summing jobs values into tract-years...')
jobs_df = pandas.DataFrame(data=None, columns=jobs_columns, index=pandas.MultiIndex.from_frame(raw_jobs_df[['GEOID','year']].drop_duplicates().sort_values(by=['GEOID','year'])))
jobs_df.loc[:,:][jobs_columns] = raw_jobs_df.groupby(['GEOID','year'])[jobs_columns].sum()

print('Extrapolating jobs for recent years needed...')
latest_job_data_year = np.max(jobs_df.index.get_level_values('year'))
for extrapolate_year in range(latest_job_data_year+1, latest_year+1):
    print('\t{0:.0f}'.format(extrapolate_year))
    new_jobs_df = jobs_df.xs(latest_job_data_year, level='year').assign(year = extrapolate_year).reset_index().set_index(['GEOID','year'])
    jobs_df = pandas.concat((jobs_df, new_jobs_df), axis=0)
jobs_df = jobs_df.sort_index()

print('calculating total and small business jobs...')
jobs_df = jobs_df.assign(total_jobs = jobs_df['C000'])
jobs_df = jobs_df.assign(sb_jobs = jobs_df['C000'] - jobs_df['CFS05'])

print('joining to loan data...')
data_gdf = data_gdf.sort_index().join(jobs_df)


e = time.time()
g = g + (e-s)
print('\nAdded jobs data for {0:,.0f} tract-years in {1:,.0f} minutes {2:,.0f} seconds!'.format(len(data_gdf), np.floor((e-s)/60), np.floor((e-s)%60)))


getting raw jobs files...
documenting job type and segment...
getting geo crosswalk file...
getting GEOIDs from crosswalk file...
selecting jobs in city...
finding columns to sum over...
summing jobs values into tract-years...
Extrapolating jobs for recent years needed...
	2022
calculating total and small business jobs...
joining to loan data...

Added jobs data for 1,480 tract-years in 1 minutes 1 seconds!


### Get loans per job

In [11]:
s = time.time()

print('Calulating loans per job (total and with firm size 0-19)...')

data_gdf = data_gdf.assign(nLoans1_per_totaljob = data_gdf['nLoans1'] / data_gdf['total_jobs'])
data_gdf = data_gdf.assign(amtLoans1_per_totaljob = data_gdf['amtLoans1'] / data_gdf['total_jobs'])
data_gdf = data_gdf.assign(nLoans100k_per_totaljob = data_gdf['nLoans100k'] / data_gdf['total_jobs'])
data_gdf = data_gdf.assign(amtLoans100k_per_totaljob = data_gdf['amtLoans100k'] / data_gdf['total_jobs'])
data_gdf = data_gdf.assign(nLoans250k_per_totaljob = data_gdf['nLoans250k'] / data_gdf['total_jobs'])
data_gdf = data_gdf.assign(amtLoans250k_per_totaljob = data_gdf['amtLoans250k'] / data_gdf['total_jobs'])
data_gdf = data_gdf.assign(nLoansToSmallest_per_totaljob = data_gdf['nLoansToSmallest'] / data_gdf['total_jobs'])
data_gdf = data_gdf.assign(amtLoansToSmallest_per_totaljob = data_gdf['amtLoansToSmallest'] / data_gdf['total_jobs'])
data_gdf = data_gdf.assign(nLoans_per_totaljob = data_gdf['nLoans'] / data_gdf['total_jobs'])
data_gdf = data_gdf.assign(amtLoans_per_totaljob = data_gdf['amtLoans'] / data_gdf['total_jobs'])
data_gdf = data_gdf.assign(nWorkingLoans_per_totaljob = data_gdf['nWorkingLoans'] / data_gdf['total_jobs'])
data_gdf = data_gdf.assign(amtWorkingLoans_per_totaljob = data_gdf['amtWorkingLoans'] / data_gdf['total_jobs'])

data_gdf = data_gdf.assign(nLoans1_per_sbjob = data_gdf['nLoans1'] / data_gdf['sb_jobs'])
data_gdf = data_gdf.assign(amtLoans1_per_sbjob = data_gdf['amtLoans1'] / data_gdf['sb_jobs'])
data_gdf = data_gdf.assign(nLoans100k_per_sbjob = data_gdf['nLoans100k'] / data_gdf['sb_jobs'])
data_gdf = data_gdf.assign(amtLoans100k_per_sbjob = data_gdf['amtLoans100k'] / data_gdf['sb_jobs'])
data_gdf = data_gdf.assign(nLoans250k_per_sbjob = data_gdf['nLoans250k'] / data_gdf['sb_jobs'])
data_gdf = data_gdf.assign(amtLoans250k_per_sbjob = data_gdf['amtLoans250k'] / data_gdf['sb_jobs'])
data_gdf = data_gdf.assign(nLoansToSmallest_per_sbjob = data_gdf['nLoansToSmallest'] / data_gdf['sb_jobs'])
data_gdf = data_gdf.assign(amtLoansToSmallest_per_sbjob = data_gdf['amtLoansToSmallest'] / data_gdf['sb_jobs'])
data_gdf = data_gdf.assign(nLoans_per_sbjob = data_gdf['nLoans'] / data_gdf['sb_jobs'])
data_gdf = data_gdf.assign(amtLoans_per_sbjob = data_gdf['amtLoans'] / data_gdf['sb_jobs'])
data_gdf = data_gdf.assign(nWorkingLoans_per_sbjob = data_gdf['nWorkingLoans'] / data_gdf['sb_jobs'])
data_gdf = data_gdf.assign(amtWorkingLoans_per_sbjob = data_gdf['amtWorkingLoans'] / data_gdf['sb_jobs'])


per_job_columns = ['nLoans1_per_totaljob', 'amtLoans1_per_totaljob', 'nLoans100k_per_totaljob']
per_job_columns += ['amtLoans100k_per_totaljob', 'nLoans250k_per_totaljob', 'amtLoans250k_per_totaljob']
per_job_columns += ['nLoansToSmallest_per_totaljob', 'amtLoansToSmallest_per_totaljob']
per_job_columns += ['nLoans_per_totaljob', 'amtLoans_per_totaljob', 'nWorkingLoans_per_totaljob']
per_job_columns += ['amtWorkingLoans_per_totaljob', 'nLoans1_per_sbjob', 'amtLoans1_per_sbjob']
per_job_columns += ['nLoans100k_per_sbjob', 'amtLoans100k_per_sbjob', 'nLoans250k_per_sbjob']
per_job_columns += ['amtLoans250k_per_sbjob', 'nLoansToSmallest_per_sbjob', 'amtLoansToSmallest_per_sbjob']
per_job_columns += ['nLoans_per_sbjob', 'amtLoans_per_sbjob', 'nWorkingLoans_per_sbjob']
per_job_columns += ['amtWorkingLoans_per_sbjob']


print('recoding zero/zero values to zero and infinite values to NaN...')
for x in data_gdf[per_job_columns]:
    data_gdf.loc[:, x] = data_gdf[x].fillna(0)
    data_gdf.loc[data_gdf[x] == np.inf, x] = np.nan

# print('backing up...')
# data_gdf_bk = data_gdf

e = time.time()
g = g + (e-s)
print('Done in {0:,.1f} seconds!'.format(e-s)) 
# data_gdf[(data_gdf['nWorkingLoans'] == 0) | (data_gdf['CFS01'] == 0)][
#     ['nWorkingLoans', 'amtWorkingLoans', 'total_jobs', 'sb_jobs', 'nWorkingLoans_per_totaljob', 'amtWorkingLoans_per_totaljob', 'nWorkingLoans_per_sbjob', 'amtWorkingLoans_per_sbjob']
# ].sample(4).T



Calulating loans per job (total and with firm size 0-19)...
recoding zero/zero values to zero and infinite values to NaN...
Done in 0.2 seconds!


## Get ACS data

In [12]:
s = time.time()

acs5_estimates_df = pandas.DataFrame()
acs5_margins_of_error_df = pandas.DataFrame()
print('Getting ACS 5-year census data...')


for thisyear in years:
    print('\t{0:.0f} estimates...'.format(thisyear))
    acs5_estimates_this_year_df = pandas.read_csv(census_dir+'{0:.0f}/data/estimates.csv'.format(thisyear), low_memory=False, encoding='utf-8')
    acs5_estimates_this_year_df = acs5_estimates_this_year_df.drop([x for x in acs5_estimates_this_year_df.columns.tolist() if 'unnamed' in x.lower()], axis=1)

    acs5_estimates_this_year_df = acs5_estimates_this_year_df.assign(year = thisyear)
    acs5_estimates_df = pandas.concat((acs5_estimates_df, acs5_estimates_this_year_df), axis=0, sort=False)
        
    print('\t{0:.0f} margins of error...'.format(thisyear))
    acs5_margins_of_error_this_year_df = pandas.read_csv(census_dir+'{0:.0f}/data/margins_of_error.csv'.format(thisyear), low_memory=False, encoding='utf-8')
    if (thisyear <= 2014):
        acs5_margins_of_error_this_year_df = acs5_margins_of_error_this_year_df.drop([x for x in acs5_margins_of_error_this_year_df.columns.tolist() if 'unnamed' in x.lower()], axis=1)
    acs5_margins_of_error_this_year_df = acs5_margins_of_error_this_year_df.assign(year = thisyear)
    acs5_margins_of_error_df = pandas.concat((acs5_margins_of_error_df, acs5_margins_of_error_this_year_df), axis=0, sort=False)

print('setting index...')
acs5_estimates_df = acs5_estimates_df.set_index(['GEOID', 'year'])
acs5_margins_of_error_df = acs5_margins_of_error_df.set_index(['GEOID', 'year'])

print('renaming columns...')
est_oldcols = acs5_estimates_df.columns[3:]
est_newcols = [x[:-1] for x in est_oldcols]
est_coldict = dict(zip(est_oldcols, est_newcols))
acs5_estimates_df = acs5_estimates_df.rename(columns=est_coldict)

merr_oldcols = acs5_margins_of_error_df.columns[3:]
merr_newcols = [x[:-1]+'_err' for x in merr_oldcols]
merr_coldict = dict(zip(merr_oldcols, merr_newcols))
acs5_margins_of_error_df = acs5_margins_of_error_df.rename(columns=merr_coldict)


print('nan-ifying missing value mask values in margins of error...')
missing_val_flag_values = []
for i in range(0, len(acs5_margins_of_error_df.columns)):
    thiscol = acs5_margins_of_error_df.columns[i]
    if (acs5_margins_of_error_df[thiscol].min() < 0):
        print('\tFixing column {0:.0f} of {1:.0f}: {2:}...'.format(i, len(acs5_margins_of_error_df.columns), thiscol))
        missing_val_flag_values += [x for x in acs5_margins_of_error_df[acs5_margins_of_error_df[thiscol] < 0][thiscol].drop_duplicates().tolist() if x not in missing_val_flag_values]
        acs5_margins_of_error_df.loc[acs5_margins_of_error_df[thiscol] < 0, thiscol] = np.nan
missing_val_flag_values = sorted(missing_val_flag_values)
print('\n')
print('\tmissing value mask values were {0:}; now nan'.format(missing_val_flag_values))

print('joining to rest of data...')
data_gdf = data_gdf.join(acs5_estimates_df, how='left', lsuffix='_data', rsuffix='_est')
data_gdf = data_gdf.rename(columns={'state_data': 'state', 'county_data': 'county'})
data_gdf = data_gdf.join(acs5_margins_of_error_df, how='left', lsuffix='_data', rsuffix='_merr')
data_gdf = data_gdf.rename(columns={'state_data': 'state', 'county_data': 'county'})
print('\tdeleting duplicate state and county columns from ACS5 data...')
data_gdf = data_gdf.drop(['state_est', 'state_merr', 'county_est', 'county_merr'], axis=1)



print('backing up...')
data_gdf_bk = data_gdf

e = time.time()
g = g + (e-s)

#print('{0:}: Kept {1:,.0f} tract-years in {2:,.0f} minutes {3:.0f} seconds!'.format(city, len(data_gdf), np.floor((e-s)/60), np.floor((e-s)%60)))
print('{0:}: Kept {1:,.0f} tract-years in {2:,.0f} minutes {3:.0f} seconds!'.format(city, len(data_gdf), np.floor(g/60), np.floor(g%60)))




Getting ACS 5-year census data...
	2022 estimates...
	2022 margins of error...
	2021 estimates...
	2021 margins of error...
	2020 estimates...
	2020 margins of error...
	2019 estimates...
	2019 margins of error...
	2018 estimates...
	2018 margins of error...
	2017 estimates...
	2017 margins of error...
	2016 estimates...
	2016 margins of error...
	2015 estimates...
	2015 margins of error...
	2014 estimates...
	2014 margins of error...
	2013 estimates...
	2013 margins of error...
	2012 estimates...
	2012 margins of error...
setting index...
renaming columns...
nan-ifying missing value mask values in margins of error...
	Fixing column 3 of 105: B01001_001_err...
	Fixing column 4 of 105: B02001_001_err...
	Fixing column 7 of 105: B08013_001_err...
	Fixing column 14 of 105: B01001_011_err...
	Fixing column 15 of 105: B01001_012_err...
	Fixing column 18 of 105: B01001_015_err...
	Fixing column 19 of 105: B01001_016_err...
	Fixing column 29 of 105: B01001_026_err...
	Fixing column 30 of 105:

# Cross-check ACS columns

What changed from year to year?

In [13]:

#cross_check_acs5_year_columns = False
# earlier = 2020
# later = 2021
# print('Getting metadata for all ACS5 columns...')
# acs5_columns = [x for x in data_gdf.columns.tolist() if (x[0] == 'B') and (x[-4:] != '_err')]

# print('\t{0:.0f}...'.format(earlier))
# acs5_metadata_earlier_df = pandas.read_csv(data_dir+'acs5/{0:.0f}/variables.csv'.format(earlier), low_memory=False, encoding='utf-8')
# # acs5_metadata_earlier_df.loc[:, 'Unique ID'] = acs5_metadata_earlier_df['Unique ID'].apply(lambda x: '{0:}_{1:}'.format(x[:-3], x[-3:]))

# print('\t{0:.0f}...'.format(later))
# acs5_metadata_later_df = pandas.read_csv(data_dir+'acs5/{0:.0f}/variables.csv'.format(later), low_memory=False, encoding='utf-8')

# print('Joining to compare...')
# df = acs5_metadata_earlier_df.merge(acs5_metadata_later_df, how='outer', on='Unique ID', suffixes=(' {0:.0f}'.format(earlier), ' {0:.0f}'.format(later)))

# print('\n')
# print('In both {0:.0f} and {1:.0f}: {2:,.0f}'.format(earlier, later, len(df[(df['Table ID {0:.0f}'.format(earlier)].notnull()) & (df['Table ID {0:.0f}'.format(later)].notnull())])))
# print('Removed from {0:.0f} and not found in {1:.0f}: {2:,.0f}'.format(earlier, later, len(df[(df['Table ID {0:.0f}'.format(earlier)].notnull()) & (df['Table ID {0:.0f}'.format(later)].isnull())])))
# print('Added in {1:.0f} was not found in {0:.0f}: {2:,.0f}'.format(earlier, later, len(df[(df['Table ID {0:.0f}'.format(earlier)].isnull()) & (df['Table ID {0:.0f}'.format(later)].notnull())])))


# df[
#     (
#         (df['Table ID {0:.0f}'.format(earlier)].isnull()) 
#         & (df['Table ID {0:.0f}'.format(later)].notnull())
#         #& (df['Unique ID'].apply(lambda x: '.5' not in x))
#      )
#    & 
#      (
#          (df['Table ID {0:.0f}'.format(later)].apply(lambda x: 'B01002' not in str(x)))
#          & (df['Table ID {0:.0f}'.format(later)].apply(lambda x: 'B17' not in str(x)))
#      )
# ].sort_values(by='Unique ID')

# #df[df['Unique ID'].apply(lambda x: 'B23' in str(x))]

if (not(cross_check_acs5_year_columns)):
    print('Not cross-checking all ACS5 columns')
else:
    print('CHANGES from 2012 to 2013:')
    print('\tremoved:')
    print('\t\tunweighted counts and response rates (tables starting with B98)...')   #df[(df['Label 2012'].isnull()) & (df['Label {0:.0f}'.format(later)].notnull())]['Table ID {0:.0f}'.format(later)].drop_duplicates()
    print('\tadded:')
    print('\t\tDetailed race (B02003, B02014-19)')   #df[((df['Label 2012'].isnull()) & (df['Label {0:.0f}'.format(later)].notnull())) & (df['Table ID {0:.0f}'.format(later)].apply(lambda x: 'B020' in str(x)))]
    print('\t\tPlace of birth excluding born at sea (B05006161)')   #df[df['Table ID {0:.0f}'.format(later)].apply(lambda x: ('B0500' in str(x)) & (re.search('\d',str(x)[-1]) != None))][['Table ID {0:.0f}'.format(later), 'Title {0:.0f}'.format(later), 'Universe {0:.0f}'.format(later)]].drop_duplicates()
    print('\t\tSex by educational attainment by race (B15002A-I)')   #df[(df['Label 2012'].isnull()) & (df['Label {0:.0f}'.format(later)].notnull()) & df['Table ID {0:.0f}'.format(later)].apply(lambda x: 'B15' in str(x))]
    print('\t\tDetailed field of bachelors degree (B15010-12)')   #df[(df['Label 2012'].isnull()) & (df['Label {0:.0f}'.format(later)].notnull()) & df['Table ID {0:.0f}'.format(later)].apply(lambda x: 'B15' in str(x))]
    print('\t\tRatio of income to poverty level (B17002) and poverty by age (B17020 main level)')   #df[(df['Label 2012'].isnull()) & (df['Label {0:.0f}'.format(later)].notnull()) & df['Table ID {0:.0f}'.format(later)].apply(lambda x: 'B17' in str(x))]
    print('\t\tDisability by age, poverty, and employment (B181080-31)')   #df[(df['Label 2012'].isnull()) & (df['Label {0:.0f}'.format(later)].notnull()) & df['Table ID {0:.0f}'.format(later)].apply(lambda x: 'B18' in str(x))]
    print('\t\tVeteran status by race+disability (B21001A-I) and poverty (B21007)') 
    print('\t\tSex by employment by race+disability (B23002A-I) and with hours (B23023)') 
    print('\t\tSex by occupation (B2401-07)')
    print('\t\tHousehold owner costs as percentage of income (B25095)')
    print('\t\tHealth insurance by race; health insurance source (B27001A-I)')
    print('\t\tSource of health insurance (B27004-18)')
    print('\t\tComputers (B28)')
    print('\t\tImputed fields - compensate for nonresponse bias (B99)')
    print('\t\tComparisons (C)')

    print('-------')

    print('CHANGES from 2013 to 2014:')
    print('\trenamed:')
    print('\t\tmedian family income: was B19113 001, now B19113001...')
    print('\tremoved:')
    print('\t\tfirst ancestry reported (B04)...')
    print('\t\tplace of birth, foreign-born, Puerto Rico (B05006PR)...')
    print('\t\twork status and disability (B23004-23)...')
    print('\t\tsex by occupation (B24)...')
    print('\t\thousehold rooms and rent (B25002-09)...')
    print('\t\tcomparisions for all above, and also...')
    print('\t\t\tcomparisons of place of birth, geo mobility (C06, C07)...')
    print('\t\t\tworkers by sex and place (C08)...')
    print('\t\t\twomen who had a birth (C13)...')
    print('\tadded:')
    print('\t\tsex by age for foreign-born population (B05013-14)...')
    print('\t\thours worked per week, full-time status (B23026-27)...')
    print('\t\thousehold income by household costs (B2509)...')
    print('\t\thandheld computers (vars B28001005-009)')
    print('\t\tunweighted counts and imputations (B98-99)...')

    print('-------')

    print('CHANGES from 2014 to 2015:')
    #print('\trenamed:')
    #print('\t\tmedian family income: was B19113 001, now B19113001...')
    print('\tremoved:')
    print('\t\tAmerican Indian, Asian, Pacific Islander by subgroup (B02005-07)...')
    print('\t\tGrandparents living with grandchildren by age (B10060-61)...')
    print('\t\tHousehold type by tenure (B11012)...')
    print('\t\tMarital status by nativity (B12005)...')
    print('\t\tMarital status by age for women (B13001)...')
    print('\t\tYear moved into house (B25129)...')
    print('\tadded:')
    print('\t\tMore options for place of birth by nativity (B05002_016 to 027)...')
    print('\t\tPlace of birth: other northern America, Puerto Rico (B05006_162 and B05006PR)...')
    print('\t\tMore options for place of birth by year of entry (B05007_095 to 106)...')
    print('\t\tLiving arrangements of adults (B09021)...')
    print('\t\tMore occupation categories for sex by occupation (B24010/20/60)...')
    print('\t\tComparison industry by occupation (C24050/60)...')
    print('\t\tRent (B25031/56/61/63/74)...')
    print('\t\tYear structure built (B25034/36/111/112/126/127)...')
    print('\t\tHome value (B25075/85)...')
    print('\t\tHomeowner costs (B25087/94/95/96/104/107/108)...')

    print('-------')

    print('CHANGES from 2015 to 2016:')
    print('\tremoved:')
    print('\t\tInternet subscriptions (B28002/B99283)...')
    print('\tadded:')
    print('\t\tMore options for language spoken at home (B16001_120-128 and B16002_015-038)...')
    print('\t\tComparisons on language spoken at home (C16001-02)...')
    print('\t\tOptions for no computer in household (B28001_010-11)...')
    print('\t\tTypes of computers in household (B28010)...')
    print('\t\tInternet subscriptions in household (B28011)...')
    print('\t\tHousing unit nonresponse rate with reasons (B98021_010)...')
    print('\t\tImputed computer type (B99282_008-009)...')


    print('-------')

    print('CHANGES from 2016 to 2017:')
    print('\tremoved:')
    print('\t\tsome options for health insurance by type (B27011_018-35 / B27012_047-98 / B27013_023-46 / B27014_023-46)...')
    print('\tremoved:')
    print('\t\tearners in family (B19122)...')
    print('\t\tgroup quarters type (B26101-220)...')
    print('\t\tcomparisons for health insurance coverage (C27)...')
    print('\t\tvoting age citizens (B29)...')


    print('-------')

    print('CHANGES from 2017 to 2018:')

    print('\tremoved:')
    print('Option for naturalized before 1980 (B05011_011)...')
    print('Options for group quarters by sex and age (B26101_215-232 / B26108_083-085 / B26201_254-298 / B26206_042-46 / B26208_104-109)...')

    print('\tadded:')
    print('\t\tfamily size by cash assistance (B19123)...')
    print('\t\tdetailed occupation (B24114-16)...')
    print('\t\tmore choices for detailed occupation (B24121/22/23/24/25/26_527-566)...')
    print('\t\tOptions for group quarters (B26114_023-35 / B26214_030-44)')


    print('-------')

    print('CHANGES from 2018 to 2019:')
    print('\tremoved:')
    print('\t\tUnweighted sample count (B00001/02)...')
    print('\t\tChildren in nonfamily households (B09005_006)...')
    print('\t\tPresence of unmarried partner (B09008)...')
    print('\t\tOptions for living situation (B09019_027-38)...')
    print('\t\tPartner in unmarried household (B13004_010-11...')

    print('\tadded:')
    print('\t\tOptions for foreign-born place of birth (B05006_163-68)...')
    print('\t\tPlace of birth by year of entry (B05015)...')
    print('\t\tCohabiting couples (B11008/09/12)...')
    print('\t\tOptions for median duration of current marriage (B12504_010-13)...')
    print('\t\tFinal interview count (B98003_001)...')

    print('-------')

    print('CHANGES from 2019 to 2020:')
    print('\tremoved: NONE')
    print('\tadded:')
    print('\tdetailed industry (B24134-6)...')
    print('\thousing units without a mortgage (B25081_009)...')
    print('\thousing unit vacancies (B25130-31)...')
    print('\t\tage by computer ownership (B28012_021)...')
    print('\t\tallocation of vacancy (B992524-5)...')


    print('-------')

    print('CHANGES from 2020 to 2021:')
    print('removed:')
    print('\tDetailed race (B02003)...')
    print('\tEducational attainment by race (B15002A-I / B15010)...')
    print('\tDetailed language (B16002-3)...')
    print('\tIncome/poverty ratio (B17002)...')
    print('\tNumber of disabilies / employment by disability (B18108/12)...')
    print('\tSex by age by employment/occupation by race (B23-4A-I)...')
    print('\tReasons for vacanting houses (B25)...')
    print('\tHealth insurance by age by race (B27A-I)...')
    print('\tVacancy allocation (B9925I)...')
    print('\tComparison for sex by age (C01001)...')
    print('\tComparison for American Indian / Asian / Pacific islander subgroup (C02014-6)...')
    print('\tComparison for ancsetry/nativity/birth/mobility (C03-7)...')
    print('\tComparison for transportation (C08-11)...')
    print('\tComparison for education/language/poverty/disability (C14-18/21/30-1)...')
    print('\tComparison for employment and disability (C21)...')
    print('\tComparison for public assistance (C22)...')
    print('\tComparison for ex by age by employment/occupation by race (C23-24)...')
    print('\tComparison for reasons for vacanting houses (C25)...')
    print('\tComparison for health insurance by age by race (C27)...')
    print('added:')
    print('\tPoverty status by housing unit (B17)...')
    print('\tElectricity costs (B25)...')
    print('\tMedian age by sex (B01002 plus A-I)...')
    print('\tPacific Islander by category (B02016/19)...')
    print('\tMedian age (B06002/07002)...')
    print('\tMoves within USA (B07101)...')
    print('\tMedian age by transportation (B08103/B08503)...')
    print('\tMFI for grandparent households (B10010)...')
    print('\tMedian age at first marriage (B12007)...')

    print('\tTotal fields of bachelors degree (B15012)...')
    print('\tQuintiles of household income (B19080-2)...')
    print('\tMedian age (B23/26)')
    print('\tCoverage rates (B98-9)')


Not cross-checking all ACS5 columns


## Count missing values in estimates

In [14]:
print('Counting missing values in {0:}...'.format(city))
this_city_geoids = data_gdf.index.get_level_values(level='GEOID').drop_duplicates().tolist()

for oneyear in years[-1::-1]:
    #print(data_gdf.xs(oneyear, level='year').columns.tolist())
    acs5_thisyear_metadata_df = pandas.read_csv('{0:}{1:.0f}/metadata/variables.csv'.format(census_dir, oneyear))
    if (oneyear <= 2014):
        acs5_thisyear_metadata_df.loc[:, 'Unique ID'] = acs5_thisyear_metadata_df['Unique ID'].apply(lambda x: '{0:}_{1:}'.format(x[:-3], x[-3:]))
    #print(acs5_thisyear_metadata_df[acs5_thisyear_metadata_df['Unique ID'].apply(lambda x: '_' not in x)])
    acs5_thisyear_metadata_df = acs5_thisyear_metadata_df.set_index('Unique ID')
    print(oneyear)
    print('-------------------')    
    acs5_thisyear_columns = acs5_thisyear_metadata_df[acs5_thisyear_metadata_df.index.isin(data_gdf.xs(oneyear, level='year').columns.tolist())].index.tolist()
    #print(acs5_thisyear_columns)
    
    
    for thiscol in acs5_thisyear_columns:
        
    
        if (len(data_gdf.xs(oneyear, level='year')[data_gdf.xs(oneyear, level='year')[thiscol].isnull()]) > 0):
            print('{0:}: {1:,.0f} missing values...'.format(thiscol, len(
                data_gdf.xs(oneyear, level='year')[(data_gdf.xs(oneyear, level='year')[thiscol].isnull()) & (data_gdf.xs(oneyear, level='year').index.isin(this_city_geoids))]
            )))
            
            
    print('-------------------')
    print('\n')
#     for thiscol in data_gdf.xs(oneyear, level='year').columns.tolist():
#         print(thiscol)
    
    
#     print(oneyear)

#     print(acs5_thisyear_metadata_df[acs5_thisyear_metadata_df.index.isin(data_gdf.xs(oneyear, level='year').columns.tolist())])
#     print('-------------------')
#     print('\n')
    
    
    
#data_gdf.columns.tolist()
pprint('k')

Counting missing values in Pittsburgh...
2012
-------------------
-------------------


2013
-------------------
-------------------


2014
-------------------
-------------------


2015
-------------------
B08013_001: 11 missing values...
B19013_001: 13 missing values...
B19013A_001: 40 missing values...
B19013B_001: 120 missing values...
B19113_001: 18 missing values...
B25035_001: 10 missing values...
B25077_001: 25 missing values...
-------------------


2016
-------------------
B08013_001: 13 missing values...
B19013_001: 11 missing values...
B19013A_001: 21 missing values...
B19013B_001: 52 missing values...
B19113_001: 15 missing values...
B25035_001: 10 missing values...
B25077_001: 15 missing values...
-------------------


2017
-------------------
B08013_001: 17 missing values...
B19013_001: 12 missing values...
B19013A_001: 22 missing values...
B19013B_001: 58 missing values...
B19113_001: 16 missing values...
B25035_001: 12 missing values...
B25077_001: 17 missing values...

### Calculate composite demographic columns

In [15]:
s = time.time()
# print('getting from backup...')
# data_gdf = data_gdf_bk

print('\ncalculating and renaming estimates columns for IVs...')

if (debug >= 1):
    print('population 25 and older with bachelors degree or higher (B15003_022 t.e.m. B15003_025...')
h = data_gdf['B15003_022'] + data_gdf['B15003_023'] + data_gdf['B15003_024']  + data_gdf['B15003_025']
data_gdf = data_gdf.assign(educated = pandas.to_numeric(h, errors='coerce'))

if (debug >= 1):
    print('...population 25 years and older...')
data_gdf = data_gdf.rename(columns={'B15003_001': 'pop_25plus'})


if (debug >= 1):
    print('...householder sex & race, unempoyment, poverty, household type, home value, home age, travel time...')
data_gdf = data_gdf.rename(columns = {  
    'B11001_006': 'female_householders',
    'B11001A_001': 'white_householders',
    'B11001B_001': 'black_householders',
    'B23025_005': 'unemployed_16plus',
    'B17001_002': 'poverty_past_12_months',
    'B25077_001': 'median_home_value',
    'B25035_001': 'median_year_built',
})

if (debug >= 1):
    print('...race, owner-occupied units, mfi, vacants...')
data_gdf = data_gdf.rename(columns = {    
    'B02001_002': 'pop_white',
    'B02001_003': 'pop_black',
    'B25003_002': 'owner_occ_housing_units',
    'B19113_001': 'mfi',
    'B25002_002': 'occupied_housing_units',
    'B25002_003': 'vacant_housing_units',
    'B11001_002': 'household_type_family',
    'B11001_007': 'household_type_nonfamily',
    'B25003_003': 'tenure_rent',
    'B25034_001': 'year_built'
})

if (debug >= 1):
    print('....comparison variables: total population, total households, poverty status, rentage...')
data_gdf = data_gdf.rename(columns = {
    'B01001_001': 'pop_total',
    'B02001_001': 'pop_by_race_total',
    'B11001_001': 'total_households',
    'B25002_001': 'total_housing_units',
    'B23025_002': 'labor_force_16plus',
    'B17001_001': 'poverty_status_known',
    'B25003_001': 'tenure_total'
})

if (debug >= 1):
    print('MFI & median home value: substituting "." with np.nan, converting to numeric...')
    print('median home value: substituting "." with np.nan, converting to numeric...')
data_gdf.loc[data_gdf['mfi'] == '.', 'mfi'] = pandas.to_numeric(data_gdf['mfi'][data_gdf['mfi'] == '.'], errors='coerce')
data_gdf.loc[data_gdf['median_home_value'] == '.', 'median_home_value'] = pandas.to_numeric(data_gdf['median_home_value'][data_gdf['median_home_value'] == '.'], errors='coerce')


# print('backing up...')
# data_gdf_bk = data_gdf

e = time.time()
g = g + (e-s)


print('Done in {0:,.0f} seconds!'.format(e-s))


# unrenamed_columns = []
# for x in data_gdf.columns:
#     if ((x[0] == 'B') and ('_err' not in x)):
#         unrenamed_columns.append(x)
# acs5_metadata_df[acs5_metadata_df['variable'].isin(unrenamed_columns)][-9:]




calculating and renaming estimates columns for IVs...
population 25 and older with bachelors degree or higher (B15003_022 t.e.m. B15003_025...
...population 25 years and older...
...householder sex & race, unempoyment, poverty, household type, home value, home age, travel time...
...race, owner-occupied units, mfi, vacants...
....comparison variables: total population, total households, poverty status, rentage...
MFI & median home value: substituting "." with np.nan, converting to numeric...
median home value: substituting "." with np.nan, converting to numeric...
Done in 0 seconds!


# Create error calculating functions

In [16]:
### Guide on how to calculate errors in percentages:
# https://www.census.gov/content/dam/Census/library/publications/2018/acs/acs_general_handbook_2018_ch08.pdf
    
## Aggregating Data Across Population Subgroups: add error for each group in quadrature, divide by 1.645 for serr
s = time.time()

def find_serr_educated(row):

    return pandas.to_numeric(np.sqrt(row['B15003_022_err']**2 + row['B15003_023_err']**2 + row['B15003_024_err']**2 + row['B15003_025_err']**2
                                ) / 1.645, errors='coerce')

# def find_serr_householders(row):
#     return pandas.to_numeric(np.sqrt(row['B11001_002_err']**2 + row['B11001_007_err']**2 
#                                 ) / 1.645, errors='coerce')

e = time.time()
g = g + (e-s)
print('Defined standard-error-calculating functions!')
print('ok')

Defined standard-error-calculating functions!
ok


# Calculate errors

In [17]:
s = time.time()

# print('getting from backup..')
# data_gdf = data_gdf_bk

if (debug >= 1):
    print('...standard errors for hs graduates 25 and older (using custom serr-finding function...')
data_gdf = data_gdf.assign(educated_serr = pandas.to_numeric(data_gdf.apply(lambda row: find_serr_educated(row), axis=1), errors='coerce'))

if (debug >= 1):
    print('...margins of error for householder sex & race, unempoyment, poverty, home value, home age...')
data_gdf = data_gdf.rename(columns = {     
    'B11001_001_err': 'total_households_err',
    'B11001_006_err': 'female_householders_err',
    'B11001A_001_err': 'black_householders_err',
    'B11001B_001_err': 'white_householders_err',
    'B23025_005_err': 'unemployed_16plus_err',
    'B17001_002_err': 'poverty_past_12_months_err',
    'B25077_001_err': 'median_home_value_err',
    'B25035_001_err': 'median_year_built_err',
    'B25003_002_err': 'owner_occ_housing_units_err',
    'B19113_001_err': 'mfi_err',
    'B25002_002_err': 'occupied_housing_units_err',
    'B25002_003_err': 'vacant_housing_units_err',
    'B11001_002_err': 'household_type_family_err',
    'B11001_007_err': 'household_type_nonfamily_err',
    'B25003_003_err': 'tenure_rent_err',
    'B15003_001_err': 'pop_25plus_err'
})

data_gdf = data_gdf.assign(household_type_total = data_gdf['household_type_family'] + data_gdf['household_type_nonfamily'])
data_gdf = data_gdf.assign(household_type_total_err = data_gdf['household_type_family_err'] + data_gdf['household_type_nonfamily_err'])

if (debug >= 1):
    print('MFI & median home value: substituting "." with np.nan, converting to numeric...')
data_gdf.loc[data_gdf['mfi_err'] == '.', 'mfi_err'] = pandas.to_numeric(data_gdf['mfi_err'][data_gdf['mfi_err'] == '.'], errors='coerce')
data_gdf.loc[data_gdf['median_home_value_err'] == '.', 'median_home_value_err'] = pandas.to_numeric(data_gdf['median_home_value_err'][data_gdf['median_home_value_err'] == '.'], errors='coerce')


print('\ncalculating and renaming margins of error for comparison variables...')
if (debug >= 1):
    print('...race, owner-occupied units, mfi...')
data_gdf = data_gdf.rename(columns = {
    'B01001_001_err': 'pop_total_err',
    'B02001_002_err': 'pop_white_err',
    'B02001_003_err': 'pop_black_err',
    'B02001_001_err': 'pop_by_race_total',
    'B17001_001_err': 'poverty_status_known_err',
    'B02001_001_err': 'pop_by_race_total_err',
    'B25002_001_err': 'total_housing_units_err',
    'B23025_002_err': 'labor_force_16plus_err',
    'B25003_001_err': 'tenure_total_err',
    
    
})

# if (debug >= 1):
#     print('...total householders...')
# data_gdf = data_gdf.assign(total_householders_serr = pandas.to_numeric(data_gdf.apply(lambda row: find_serr_householders(row), axis=1), errors='coerce'))


#print('dropping columns we do not care about...')
# columns_do_not_care = []
# columns_do_not_care += ['B11001_002', 'B11001_007']
# columns_do_not_care += ['B11001_002_err', 'B11001_007_err']
# columns_do_not_care += ['B01001_048_err','B01001_049_err']#,'STATE']
#data_gdf = data_gdf.drop(columns_do_not_care, axis=1)


# print('Calculated errors for all columns!')
# print('backing up...')
# data_gdf_bk = data_gdf

e = time.time()
g = g + (e-s)

unrenamed_columns_with_errs = []

for x in data_gdf.columns:
#    print(x)
    if ((x[0] == 'B') and ('_err' in x)):
        unrenamed_columns_with_errs.append(x)

unrenamed_columns = []        
for y in unrenamed_columns_with_errs:
    unrenamed_columns.append(y[:-4])
#acs5_metadata_df[acs5_metadata_df['variable'].isin(unrenamed_columns)][80:]
#print(unrenamed_columns)

e = time.time()
print('Done in {0:,.0f} seconds!'.format(e-s))

...standard errors for hs graduates 25 and older (using custom serr-finding function...
...margins of error for householder sex & race, unempoyment, poverty, home value, home age...
MFI & median home value: substituting "." with np.nan, converting to numeric...

calculating and renaming margins of error for comparison variables...
...race, owner-occupied units, mfi...
Done in 0 seconds!


# Prepare to percentify

In [18]:
s = time.time()
vars_for_percentification = ['pop_white', 'pop_black', 'black_householders', 'white_householders']
vars_for_percentification += ['owner_occ_housing_units', 'educated', 'female_householders']
vars_for_percentification += ['unemployed_16plus', 'poverty_past_12_months']
vars_for_percentification += ['tenure_rent', 'household_type_family']

vars_for_percentification += ['pop_white_err', 'pop_black_err', 'black_householders_err', 'white_householders_err']
vars_for_percentification += ['owner_occ_housing_units_err', 'educated_serr', 'female_householders_err']
vars_for_percentification += ['unemployed_16plus_err', 'poverty_past_12_months_err']
vars_for_percentification += ['tenure_rent_err', 'household_type_family_err']

vars_for_percentification += ['pop_total', 'total_householders', 'pop_by_race_total', 'pop_25plus', 'labor_force_16plus']
vars_for_percentification += ['poverty_status_known', 'vacant_housing_units', 'total_housing_units']
vars_for_percentification += ['tenure_total', 'household_type_total']

vars_for_percentification += ['pop_totatl_err', 'total_householders_serr', 'pop_by_race_total_err', 'pop_25plus_err', 'labor_force_16plus_err']
vars_for_percentification += ['poverty_status_known_err', 'vacant_housing_units_err', 'total_housing_units_err']
vars_for_percentification += ['tenure_total_err', 'household_type_total_err']
#vars_for_percentification
#city_tracts_years_df[vars_for_percentification].columns.tolist()
e = time.time()
g = g + (e-s)

print('ok')

ok


# Calculate percentages for needed demographic variables

In [19]:
s = time.time()

data_gdf = data_gdf.assign(pct_white = pandas.to_numeric((data_gdf['pop_white'] / data_gdf['pop_by_race_total']), errors='coerce'))
data_gdf = data_gdf.assign(pct_black = pandas.to_numeric((data_gdf['pop_black'] / data_gdf['pop_by_race_total']), errors='coerce'))

data_gdf = data_gdf.assign(pct_white_householders = pandas.to_numeric((data_gdf['white_householders'] / data_gdf['total_households']), errors='coerce'))
data_gdf = data_gdf.assign(pct_black_householders = pandas.to_numeric((data_gdf['black_householders'] / data_gdf['total_households']), errors='coerce'))
data_gdf = data_gdf.assign(pct_female_householders = pandas.to_numeric((data_gdf['female_householders'] / data_gdf['total_households']), errors='coerce'))

data_gdf = data_gdf.assign(peducated = pandas.to_numeric(data_gdf['educated'], errors='coerce') / pandas.to_numeric(data_gdf['pop_25plus'], errors='coerce'))
data_gdf = data_gdf.assign(pct_unemployed = pandas.to_numeric(data_gdf['unemployed_16plus'], errors='coerce') / pandas.to_numeric(data_gdf['labor_force_16plus'], errors='coerce'))
data_gdf = data_gdf.assign(pct_poverty = pandas.to_numeric(data_gdf['poverty_past_12_months'], errors='coerce') / pandas.to_numeric(data_gdf['poverty_status_known'], errors='coerce'))
data_gdf = data_gdf.assign(pct_vacant = pandas.to_numeric(data_gdf['vacant_housing_units'], errors='coerce') / pandas.to_numeric(data_gdf['total_housing_units'], errors='coerce'))

data_gdf = data_gdf.assign(pct_rent = pandas.to_numeric(data_gdf['tenure_rent'], errors='coerce') / pandas.to_numeric(data_gdf['tenure_total'], errors='coerce'))
data_gdf = data_gdf.assign(pct_household_family = pandas.to_numeric(data_gdf['household_type_family'], errors='coerce') / pandas.to_numeric(data_gdf['household_type_total'], errors='coerce'))

e = time.time()
g = g + (e-s)
print('ok')
#data_gdf.columns.tolist()

data_gdf.sample(1).T



ok


GEOID,14000US42003262000
year,2022
STATEFP,42
county,3
census_tract,2620
NAME,2620
NAMELSAD,Census Tract 2620
...,...
pct_unemployed,0.0773639
pct_poverty,0.296055
pct_vacant,0.126631


# Functions to calculate errors in percentages

In [20]:
#Guide on how to do this:
#### https://www.census.gov/content/dam/Census/library/publications/2018/acs/acs_general_handbook_2018_ch08.pdf

# X and Y are the measured values (not the errors) - X for the subsgroup and Y for the whole sample
# Let P = X/Y  (the proportion we calculated in the last step)
# dX and dY are the measured errors
# dP = (1/Y) * np.sqrt(dX**2 - (P**2 * dY**2))
# Standard error of P is dP/1.645
#### this calculation is done verbosely in fnid_pop_white_serr, quickly in other functions

s = time.time()
def find_errors_in_pct(X, Y, dX, dY, verboselevel = 0):
    try:
        P = X / Y
        oneoverY = 1 / Y
        dXsq = dX**2
        dYsq = dY**2
        Psq = P**2
        PsqdYsq = Psq * dYsq
        if (PsqdYsq <= dXsq):
            underroot = dXsq - PsqdYsq
        else:
            underroot = dXsq + PsqdYsq
        rooty = np.sqrt(underroot)
        dP = oneoverY * rooty
        SE = dP / 1.645
        if (verboselevel >= 2):
#            print('X = pop_white, Y = pop_total')
            print('X = {0:.0f}, dX = {1:.0f} ({2:.1%} error)'.format(X, dX, dX/X))
            print('Y = {0:.0f}, dY = {1:.0f} ({2:.1%} error)'.format(Y, dY, dY/Y))
        if (verboselevel >= 3):
            print('P = {0:.3f}'.format(P))
            print('dXsq = {0:.0f}, dYsq = {1:.0f}, Psq = {2:.3f}'.format(dXsq, dYsq, Psq))
            print('PsqdYsq = {0:.0f}, underroot = {1:.0f}, rooty = {2:.3f}'.format(PsqdYsq, underroot, rooty))
            print('dP = {0:.3f}'.format(dP))
            print('SE = {0:.3f}'.format(SE))
        if (verboselevel >= 2):
            print('RESULT: {0:.2%} +/- {1:.2%}'.format(P, SE)) 
            print('\n')
        return pandas.to_numeric(SE, errors='coerce')
    except ZeroDivisionError:
        return np.nan
    
e = time.time()
g = g + (e-s)    
print('Defined functions to calculate standard errors in percentages!')
print('ok')

Defined functions to calculate standard errors in percentages!
ok


# Calculate errors in percntages

In [21]:
verboselevel = 0
s = time.time()

print('Calculating errors in percentages...')
data_gdf = data_gdf.assign(pct_white_serr = np.nan)
data_gdf = data_gdf.assign(pct_black_serr = np.nan)
data_gdf = data_gdf.assign(pct_white_householders_serr = np.nan)
data_gdf = data_gdf.assign(pct_black_householders_serr = np.nan)
data_gdf = data_gdf.assign(pct_female_householders_serr = np.nan)
data_gdf = data_gdf.assign(peducated_serr = np.nan)
data_gdf = data_gdf.assign(pct_unemployed_serr = np.nan)
data_gdf = data_gdf.assign(pct_poverty_serr = np.nan)
data_gdf = data_gdf.assign(pct_vacant_serr = np.nan)


data_gdf = data_gdf.assign(pct_rent_serr = np.nan)
data_gdf = data_gdf.assign(pct_household_family_serr = np.nan)


#data_gdf.loc[:, 
#              'poverty_status_known_last12months_total_err'] = pandas.to_numeric(data_gdf['poverty_status_known_last12months_total_err'], errors='coerce')


for ix, thisrow in data_gdf.iterrows():
    if (verboselevel >= 2):
        print('Census tract {0:}...'.format(ix))
    #print('pct_white_serr...')
    data_gdf.loc[ix, 'pct_white_serr'] = find_errors_in_pct(thisrow['pop_white'], thisrow['pop_by_race_total'], thisrow['pop_white_err'], thisrow['pop_by_race_total_err'], verboselevel)
    #print('pct_black_serr...')
    data_gdf.loc[ix, 'pct_black_serr'] = find_errors_in_pct(thisrow['pop_black'], thisrow['pop_by_race_total'], thisrow['pop_black_err'], thisrow['pop_by_race_total_err'], verboselevel)
    #print('pct_white_householders_serr...')
    data_gdf.loc[ix, 'pct_white_householders_serr'] = find_errors_in_pct(thisrow['white_householders'], thisrow['total_households'], thisrow['white_householders_err'], thisrow['total_households_err'], verboselevel)
    data_gdf.loc[ix, 'pct_black_householders_serr'] = find_errors_in_pct(thisrow['white_householders'], thisrow['total_households'], thisrow['white_householders_err'], thisrow['total_households_err'], verboselevel)
    data_gdf.loc[ix, 'pct_female_householders_serr'] = find_errors_in_pct(thisrow['white_householders'], thisrow['total_households'], thisrow['white_householders_err'], thisrow['total_households_err'], verboselevel)
    
    data_gdf.loc[ix, 'peducated_serr'] = find_errors_in_pct(thisrow['educated'], thisrow['pop_25plus'], thisrow['educated_serr'], thisrow['pop_25plus_err'], verboselevel)
    data_gdf.loc[ix, 'pct_unemployed_serr'] = find_errors_in_pct(thisrow['unemployed_16plus'], thisrow['unemployed_16plus_err'], thisrow['labor_force_16plus_err'], thisrow['labor_force_16plus_err'], verboselevel)
    data_gdf.loc[ix, 'pct_poverty_serr'] = find_errors_in_pct(thisrow['poverty_past_12_months'], thisrow['poverty_status_known'], thisrow['poverty_past_12_months_err'], thisrow['poverty_status_known_err'], verboselevel)
    data_gdf.loc[ix, 'pct_vacant_serr'] = find_errors_in_pct(thisrow['vacant_housing_units'], thisrow['total_housing_units'], thisrow['vacant_housing_units_err'], thisrow['total_housing_units_err'], verboselevel)

    data_gdf.loc[ix, 'pct_rent_serr'] = find_errors_in_pct(thisrow['tenure_rent'], thisrow['tenure_total'], thisrow['tenure_rent_err'], thisrow['tenure_total_err'], verboselevel)
    data_gdf.loc[ix, 'pct_household_family_serr'] = find_errors_in_pct(thisrow['household_type_family'], thisrow['household_type_total'], thisrow['household_type_family_err'], thisrow['household_type_total_err'], verboselevel)

if (verboselevel >= 1):
    for ix, thisrow in data_gdf.iterrows():
        print('Census tract {0:,.0f}'.format(ix))
        print('{0:,.0f} +/- {1:,.0f} white'.format(
            thisrow['pop_white'], thisrow['pop_white_err']
        ))
        print('{0:,.0f} +/- {1:,.0f} total'.format(
            thisrow['pop_total'], thisrow['pop_total_err']
        ))
        print('{0:.1%} +/- {1:.1%}'.format(
            thisrow['pct_white'], thisrow['pct_white_serr']
        ))
        print('\n')

# print('backing up...')
# data_gdf_bk = data_gdf
    
e = time.time()
g = g + (e-s)
print('ok')

Calculating errors in percentages...
ok


# Correct for inflation

## Get inflation values from Board of Labor Statistics Consumer Price Index data

In [22]:
s = time.time()
baseline_year = latest_year

inflator_df = pandas.DataFrame(data=None, columns=['theyear','themonth','thevalue'])

headers = {'Content-type': 'application/json'}
data = json.dumps({"seriesid": ['CUSR0000SA0'],"startyear":earliest_year, "endyear":latest_year})  # All items in U.S. city average, all urban consumers, seasonally adjusted: https://www.bls.gov/cpi/tables/supplemental-files/historical-cpi-u-202007.pdf
p = requests.post('https://api.bls.gov/publicAPI/v2/timeseries/data/', data=data, headers=headers)
json_data = json.loads(p.text)

if (json_data['status'] == 'REQUEST_NOT_PROCESSED'):
    print('Request not processed, setting manually...')
    inflator_df.loc[0, ['theyear', 'themonth', 'thevalue']] = [2021, 1, 262.518]
    inflator_df.loc[1, ['theyear', 'themonth', 'thevalue']] = [2020, 1, 258.906]
    inflator_df.loc[2, ['theyear', 'themonth', 'thevalue']] = [2019, 1, 251.712]
    inflator_df.loc[3, ['theyear', 'themonth', 'thevalue']] = [2018, 1, 247.867]
    inflator_df.loc[4, ['theyear', 'themonth', 'thevalue']] = [2017, 1, 242.839]
    inflator_df.loc[5, ['theyear', 'themonth', 'thevalue']] = [2016, 1, 237.652]
    inflator_df.loc[6, ['theyear', 'themonth', 'thevalue']] = [2015, 1, 234.747]
    inflator_df.loc[7, ['theyear', 'themonth', 'thevalue']] = [2014, 1, 235.288]
    inflator_df.loc[8, ['theyear', 'themonth', 'thevalue']] = [2013, 1, 231.679]
    inflator_df.loc[9, ['theyear', 'themonth', 'thevalue']] = [2012, 1, 227.842]
#     inflator_df.loc[10, ['theyear', 'themonth', 'thevalue']] = [2011, 1, 221.187]
#     inflator_df.loc[11, ['theyear', 'themonth', 'thevalue']] = [2010, 1, 217.488]
else:
    cnt = 0
    for series in json_data['Results']['series']:
        seriesId = series['seriesID']
        for item in series['data']:
            inflator_df.loc[cnt, ['theyear', 'themonth', 'thevalue']] = [int(item['year']), int(item['period'][1:]), float(item['value'])]
            cnt += 1 

## 2022 not included because reasons, add it manually here        
if (json_data['status'] == 'REQUEST_NOT_PROCESSED'):
    inflator_df.loc[inflator_df.index.max()+1, ['theyear', 'themonth', 'thevalue']] = [2022, 1, 282.390]
else:
    inflator_df.loc[inflator_df.index.max()+1, ['theyear', 'themonth', 'thevalue']] = [2022, 1, 282.390]
    inflator_df.loc[inflator_df.index.max()+1, ['theyear', 'themonth', 'thevalue']] = [2022, 2, 284.535]
    inflator_df.loc[inflator_df.index.max()+1, ['theyear', 'themonth', 'thevalue']] = [2022, 3, 287.553] 
    inflator_df.loc[inflator_df.index.max()+1, ['theyear', 'themonth', 'thevalue']] = [2022, 4, 288.764]
    inflator_df.loc[inflator_df.index.max()+1, ['theyear', 'themonth', 'thevalue']] = [2022, 5, 291.359]
    inflator_df.loc[inflator_df.index.max()+1, ['theyear', 'themonth', 'thevalue']] = [2022, 6, 294.996]
    inflator_df.loc[inflator_df.index.max()+1, ['theyear', 'themonth', 'thevalue']] = [2022, 7, 294.977]
    inflator_df.loc[inflator_df.index.max()+1, ['theyear', 'themonth', 'thevalue']] = [2022, 8, 295.209]
    inflator_df.loc[inflator_df.index.max()+1, ['theyear', 'themonth', 'thevalue']] = [2022, 2, 284.535]
    inflator_df.loc[inflator_df.index.max()+1, ['theyear', 'themonth', 'thevalue']] = [2022, 9, 296.341]
    inflator_df.loc[inflator_df.index.max()+1, ['theyear', 'themonth', 'thevalue']] = [2022, 10, 297.863]
    inflator_df.loc[inflator_df.index.max()+1, ['theyear', 'themonth', 'thevalue']] = [2022, 11, 298.648]
    inflator_df.loc[inflator_df.index.max()+1, ['theyear', 'themonth', 'thevalue']] = [2022, 12, 298.812]
    
inflator_df = inflator_df.sort_values(by=['theyear', 'themonth'], ascending=False).reset_index(drop=True)
baseline_value = inflator_df[inflator_df['theyear'] == baseline_year]['thevalue'].values[0]
inflator_df = inflator_df.assign(thefactor = baseline_value / inflator_df['thevalue'])
inflator_df = inflator_df.set_index('theyear')
inflator_df = inflator_df[inflator_df['themonth'] == 1]   # inflate by year by choosing only January

e = time.time()
g = g + (e-s)
# #inflator_df[inflator_df['themonth'] == 1]

inflator_df

,themonth,thevalue,thefactor
theyear,,,
2022,1,282.39,1.05815
2021,1,262.518,1.13825
2020,1,258.906,1.15413
2019,1,252.561,1.18313
2018,1,248.859,1.20073
2017,1,243.618,1.22656
2016,1,237.652,1.25735
2015,1,234.747,1.27291
2014,1,235.288,1.26998


In [23]:
s = time.time()

# print('getting from backup...')
# data_gdf = data_gdf_bk

money_columns = []
money_columns += ['amtLoans1', 'amtLoans100k', 'amtLoans250k', 'amtLoansToSmallest']
money_columns += ['avgSmallLoan', 'amtLoans', 'amtWorkingLoans']
money_columns += ['amtLoans1_per_totaljob', 'amtLoans100k_per_totaljob', 'amtLoans250k_per_totaljob']
money_columns += ['amtLoansToSmallest_per_totaljob', 'amtLoans_per_totaljob', 'amtWorkingLoans_per_totaljob']
money_columns += ['amtLoans1_per_sbjob', 'amtLoans100k_per_sbjob', 'amtLoans250k_per_sbjob']
money_columns += ['amtLoansToSmallest_per_sbjob', 'amtLoans_per_sbjob', 'amtWorkingLoans_per_sbjob']
money_columns += ['mfi', 'median_home_value', 'mfi_err', 'median_home_value_err']



adjusted_money_df = pandas.DataFrame(data=None, columns=['GEOID']+money_columns).set_index('GEOID')

for thisyear in years:
    thefactor = inflator_df.loc[thisyear]['thefactor']
    adjusted_money_df_i = data_gdf.xs(thisyear, level='year')[money_columns].apply(lambda x: x*thefactor)
    adjusted_money_df_i = adjusted_money_df_i.assign(year = thisyear).reset_index()
    adjusted_money_df = pandas.concat((adjusted_money_df, adjusted_money_df_i), axis=0)

for x in money_columns:
    adjusted_money_df = adjusted_money_df.rename(columns={x: x+'_adj'})
# adjusted_money_df = adjusted_money_df.reset.set_index(['GEOID', 'year'])
    
# # data_gdf = pandas.concat((data_gdf, adjusted_money_df), axis=1)
#.set_index(['GEOID', 'year'])
adjusted_money_df = adjusted_money_df.set_index(['GEOID', 'year'])
data_gdf = pandas.concat((data_gdf,  adjusted_money_df), axis=1)

e = time.time()
g = g + (e-s)

print('{0:}: Kept {1:,.0f} tract-years in {2:,.0f} minutes {3:.0f} seconds!'.format(city, len(data_gdf), np.floor((e-s)/60), np.floor((e-s)%60)))
#data_gdf.reset_index().groupby(['GEOID', 'year']).size().sort_values()
#adjusted_money_df.xs(2012, level='year')
#adjusted_money_df.reset_index().groupby(['GEOID', 'year']).size().sort_values(ascending=False)

#data_gdf.reset_index().groupby(['GEOID', 'year']).size().sort_values(ascending=False)




Pittsburgh: Kept 1,480 tract-years in 0 minutes 0 seconds!


# Add community statistical areas (Baltimore only)

In [24]:
s = time.time()
if (city == 'Baltimore'):
    tract_to_csa_df = pandas.read_csv(code_lookup_dir+'census_tract_to_neighborhood.csv')
    tract_to_csa_df.loc[:, 'GEOID10'] = tract_to_csa_df['GEOID10'].apply(lambda x: '14000US'+str(x))
    data_gdf = data_gdf.reset_index().merge(tract_to_csa_df[['GEOID10','TRACTCE10','NAME10','CSA2010']], left_on='GEOID', right_on='GEOID10').set_index(['GEOID', 'year'])

#data_gdf = data_gdf.drop(['GEOID10', 'TRACTCE10'], axis=1)

print('backing up...')
data_gdf_bk = data_gdf

e = time.time()
g = g + (e-s)
print('Done!')


backing up...
Done!


# Get only the columns we need, in the right order

In [25]:
s = time.time()

print('getting from backup...')
data_gdf = geopandas.GeoDataFrame()
data_gdf = data_gdf_bk

data_gdf = data_gdf.assign(city_name = city)
data_gdf = data_gdf.drop('state', axis=1)
data_gdf = data_gdf.rename(columns={'STATEFP': 'state'})
#data_gdf = data_gdf.drop(['tract_data', 'tract_merr', 'GEOID10', 'TRACTCE10'], axis=1)

# print('renaming variables describing this tract\'s state...')
# data_gdf = data_gdf.rename(columns = {'STATE': 'state_acs5'})#, 'state': 'state_loans'})#, 'STATEFP': 'state'})
# #data_gdf = data_gdf.rename(columns = {'state': 'state_loans', 'STATEFP': 'state'})

# data_gdf = data_gdf.assign(city_name = city)

print('selecting column order...')
new_columns = []
new_columns += ['state', 'county', 'census_tract']
new_columns += ['city_name', 'msa'] # city name, metropolitan statistical area (MSA) number
new_columns += ['loan_type', 'action_taken_type'] # loan type (always 4 for business) and action taken (always 1 for origination)
new_columns += ['income_group_code'] # income group code (1 to 14 or 101 to 105)
new_columns += ['income_group', 'cra_level'] # income group and CRA level (human-readable)
new_columns += ['nLoans1','amtLoans1','nLoans100k','amtLoans100k','nLoans250k','amtLoans250k','nLoansToSmallest','amtLoansToSmallest'] # number and amount of loans (directly from CRA data)
new_columns += ['nLoans', 'amtLoans', 'avgSmallLoan', 'nWorkingLoans', 'amtWorkingLoans'] # calculated total and working loans
new_columns += ['total_jobs', 'sb_jobs'] # total and small business jobs

# all jobs columns under original names
new_columns += ['C000','CA01','CA02','CA03','CE01','CE02','CE03']
new_columns += ['CNS01','CNS02','CNS03','CNS04','CNS05','CNS06','CNS07','CNS08','CNS09','CNS10','CNS11','CNS12','CNS13','CNS14','CNS15','CNS16','CNS17','CNS18','CNS19','CNS20']
new_columns += ['CR01','CR02','CR03','CR04','CR05','CR07','CT01','CT02','CD01','CD02','CD03','CD04','CS01','CS02']
new_columns += ['CFA01','CFA02','CFA03','CFA04','CFA05','CFS01','CFS02','CFS03','CFS04','CFS05']

# loans per job
new_columns += ['nLoans1_per_totaljob','amtLoans1_per_totaljob','nLoans100k_per_totaljob','amtLoans100k_per_totaljob']
new_columns += ['nLoans250k_per_totaljob','amtLoans250k_per_totaljob','nLoansToSmallest_per_totaljob','amtLoansToSmallest_per_totaljob']
new_columns += ['nLoans_per_totaljob','amtLoans_per_totaljob','nWorkingLoans_per_totaljob','amtWorkingLoans_per_totaljob']
new_columns += ['nLoans1_per_sbjob','amtLoans1_per_sbjob','nLoans100k_per_sbjob','amtLoans100k_per_sbjob','nLoans250k_per_sbjob','amtLoans250k_per_sbjob']
new_columns += ['nLoansToSmallest_per_sbjob','amtLoansToSmallest_per_sbjob','nLoans_per_sbjob','amtLoans_per_sbjob','nWorkingLoans_per_sbjob','amtWorkingLoans_per_sbjob']

# census count estimates
new_columns += ['pop_total','pop_by_race_total','pop_white','pop_black']
new_columns += ['total_households','white_householders','black_householders','female_householders']
new_columns += ['total_housing_units','occupied_housing_units','vacant_housing_units']
new_columns += ['educated','unemployed_16plus','poverty_past_12_months','mfi'] # high school graduates (ages 25+), unemployed, in poverty, MFI, travel time to work
new_columns += ['owner_occ_housing_units','median_home_value','year_built', 'median_year_built'] # home
new_columns += ['pop_25plus','labor_force_16plus','poverty_status_known']
new_columns += ['household_type_family', 'household_type_nonfamily','household_type_total', 'tenure_rent', 'tenure_total']


# census count errors
new_columns += ['pop_total_err','pop_by_race_total_err','pop_white_err','pop_black_err']
new_columns += ['total_households_err','white_householders_err','black_householders_err','female_householders_err']
new_columns += ['total_housing_units_err','occupied_housing_units_err','vacant_housing_units_err']
new_columns += ['educated_serr','unemployed_16plus_err','poverty_past_12_months_err','mfi_err'] # high school graduates (ages 25+), unemployed, in poverty, MFI, travel time to work
new_columns += ['owner_occ_housing_units_err','median_home_value_err', 'median_year_built_err'] # home
new_columns += ['pop_25plus_err','labor_force_16plus_err','poverty_status_known_err']
new_columns += ['household_type_family_err', 'household_type_nonfamily_err', 'household_type_total_err', 'tenure_rent_err', 'tenure_total_err']

# census percentages and their errors
new_columns += ['pct_white','pct_black','pct_white_householders','pct_black_householders','pct_female_householders']
new_columns += ['peducated','pct_unemployed','pct_poverty','pct_vacant', 'pct_household_family', 'pct_rent']
new_columns += ['pct_white_serr','pct_black_serr','pct_white_householders_serr','pct_black_householders_serr','pct_female_householders_serr']
new_columns += ['peducated_serr','pct_unemployed_serr','pct_poverty_serr','pct_vacant_serr', 'pct_household_family_serr', 'pct_rent_serr']

# # inflation-adjusted money values
new_columns += ['amtLoans1_adj','amtLoans100k_adj','amtLoans250k_adj','amtLoansToSmallest_adj']
new_columns += ['avgSmallLoan_adj','amtLoans_adj','amtWorkingLoans_adj']
new_columns += ['amtLoans1_per_totaljob_adj','amtLoans100k_per_totaljob_adj','amtLoans250k_per_totaljob_adj','amtLoansToSmallest_per_totaljob_adj']
new_columns += ['amtLoans_per_totaljob_adj','amtWorkingLoans_per_totaljob_adj']
new_columns += ['amtLoans1_per_sbjob_adj','amtLoans100k_per_sbjob_adj','amtLoans250k_per_sbjob_adj','amtLoansToSmallest_per_sbjob_adj','amtLoans_per_sbjob_adj','amtWorkingLoans_per_sbjob_adj']
new_columns += ['mfi_adj','median_home_value_adj','mfi_err_adj','median_home_value_err_adj']

## geographic information about the census tract
new_columns += ['NAME', 'NAMELSAD', 'MTFCC', 'FUNCSTAT', 'ALAND', 'AWATER', 'INTPTLAT', 'INTPTLON']

## more info from loans data
new_columns += ['split_county_indicator', 'population_classification', 'split_county_indicator', 'population_classification']#, 'geography_name_loans_jobs']

## ACS5 raw census data under original variable names
new_columns += [x for x in data_gdf.columns if (x not in new_columns) and (x[0] == 'B')]
## ACS5 tract metadata
#data_gdf = data_gdf.rename(columns =  {'Geography Name_acs5': 'geography_name_acs5'})
#new_columns += ['FILEID', 'FILETYPE', 'STUSAB', 'CHARITER', 'SEQUENCE', 'LOGRECNO']#, 'state_acs5',  'Name']#, 'geography_name_acs5']

# # if (city == 'Baltimore'):
# #     new_columns += ['GEOID10', 'TRACTCE10', 'NAME10', 'CSA2010']

# as traditional, geometry goes last
#new_columns += ['geometry']

# # # check that we got them all
data_gdf = data_gdf[new_columns]
# # #pprint([x for x in data_gdf.columns if x not in new_columns])


e = time.time()
g = g + (e-s)
print('Got {0:,.0f} columns for {1:,.0f} tract-years in {2:,.0f} minutes {3:,.0f} seconds!'.format(data_gdf.shape[1], data_gdf.shape[0], np.floor((e-s)/60), np.floor((e-s)%60)))
# Got 365 columns for 1,600 tract-years in 0 minutes 0 seconds!


getting from backup...
selecting column order...
Got 365 columns for 1,480 tract-years in 0 minutes 0 seconds!


In [26]:
print('outputting data...')

# print('outputting without shapefile data...')
# s = time.time()
city_name_for_output = city.replace(".","").replace(" ","_").lower()
outfilename = '{0:}_{1:.0f}_{2:.0f}_year_by_year.csv'.format(city_name_for_output, earliest_year, latest_year)



data_gdf[new_columns].to_csv(output_data_dir+outfilename)

print('\tWrote {0:,.0f} tract-years by {1:,.0f} columns (without geometry data) to:\n\t\t{2:}'.format(
    data_gdf[new_columns].shape[0], data_gdf[new_columns].shape[1], output_data_dir+outfilename
     ))
print('\n')

# print('outputting shapefile data separately...')
# outfilename_shapefiles_geo = '{0:}_{1:.0f}_{2:.0f}_shapefiles.shp'.format(city_name_for_output, earliest_year, latest_year)
# shapefile_writer_gdf = geopandas.GeoDataFrame(data=data_gdf['geometry'], columns=['geometry'], crs=data_gdf.crs, geometry='geometry')
# shapefile_writer_gdf.to_file(output_data_dir+outfilename_shapefiles_geo)

# print('\tWrote {0:,.0f} tract-years by {1:,.0f} columns (SHAPEFILE DATA ONLY) to \n\t{2:}'.format(shapefile_writer_gdf.shape[0], shapefile_writer_gdf.shape[1], output_data_dir+outfilename_shapefiles_geo))

e = time.time()
g = g + (e-s)
print('Saved {0:,.0f} variables for {1:,.0f} tract-years in {2:,.0f} minutes {3:,.0f} seconds!'.format(data_gdf.shape[1], data_gdf.shape[0], np.floor((g)/60), np.floor((g)%60)))



outputting data...
	Wrote 1,480 tract-years by 369 columns (without geometry data) to:
		/home/idies/workspace/21cc/raddick/community_reinvestment_act/city_data/pittsburgh_2012_2022_year_by_year.csv


Saved 365 variables for 1,480 tract-years in 6 minutes 24 seconds!


In [27]:
df = pandas.read_csv(output_data_dir+outfilename)
df

# Baltimore: Saved 365 variables for 2,194 tract-years in 5 minutes 1 seconds!
# Washington DC: Saved 365 variables for 2,050 tract-years in 4 minutes 51 seconds!
# Detroit: Saved 365 variables for 3,195 tract-years in 6 minutes 28 seconds!
# Cleveland: Saved 365 variables for 1,893 tract-years in 6 minutes 33 seconds!
# Newark: Saved 365 variables for 960 tract-years in 5 minutes 10 seconds!
# St. Louis: Saved 365 variables for 1,160 tract-years in 5 minutes 1 seconds!
# Richmond: Saved 365 variables for 753 tract-years in 5 minutes 10 seconds!
# San Francisco: Saved 365 variables for 2,308 tract-years in 9 minutes 15 seconds!
# Philadelphia: Saved 365 variables for 4,296 tract-years in 7 minutes 31 seconds!
# Pittsburgh: Saved 365 variables for 1,480 tract-years in 6 minutes 24 seconds!

,GEOID,year,state,county,census_tract,city_name,msa,loan_type,action_taken_type,income_group_code,...,B19013_001_err,B19013A_001_err,B19013B_001_err,B25003A_001_err,B25003A_002_err,B25003A_003_err,B25003B_001_err,B25003B_002_err,B25003B_003_err,B25034_001_err
0,14000US42003010300,2012,42,3,103.0,Pittsburgh,38300.0,4.0,1.0,3.0,...,27810.0,31273.0,7064.0,68,59,38,30,19,21,82
1,14000US42003010300,2013,42,3,103.0,Pittsburgh,38300.0,4.0,1.0,3.0,...,21260.0,20932.0,15990.0,73,69,40,24,16,19,60
2,14000US42003010300,2014,42,3,103.0,Pittsburgh,38300.0,4.0,1.0,3.0,...,8976.0,9315.0,35325.0,54,51,36,26,11,21,49
3,14000US42003010300,2015,42,3,103.0,Pittsburgh,38300.0,4.0,1.0,3.0,...,13372.0,NaN,NaN,39,28,32,35,14,28,31
4,14000US42003010300,2016,42,3,103.0,Pittsburgh,38300.0,4.0,1.0,3.0,...,16306.0,11616.0,NaN,39,29,39,27,12,30,24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1475,14000US42003982200,2018,42,3,9822.0,Pittsburgh,38300.0,4.0,1.0,14.0,...,111211.0,NaN,NaN,6,6,10,10,10,10,23
1476,14000US42003982200,2019,42,3,9822.0,Pittsburgh,38300.0,4.0,1.0,14.0,...,NaN,NaN,NaN,10,6,6,10,10,10,10
1477,14000US42003982200,2020,42,3,9822.0,Pittsburgh,38300.0,4.0,1.0,14.0,...,NaN,NaN,NaN,8,6,8,11,11,11,9
1478,14000US42003982200,2021,42,3,9822.0,Pittsburgh,38300.0,4.0,1.0,14.0,...,NaN,NaN,NaN,9,5,7,11,11,11,9
